# 第1章: グラフの基本要素

## 準備

以下のセルを順番に実行して、演習に必要な環境をセットアップします。

### ライブラリのインストール

このセルは、LangGraphおよび関連するLangChainライブラリをインストールします。実行には数分かかる場合があります。
ご利用になるLLMプロバイダーに応じて、コメントアウトを解除して必要なライブラリをインストールしてください。

In [ ]:
# === ライブラリのインストール ===
# 基本ライブラリ (LangGraphとLangChain Core)
!pip install -qU langchain langgraph

In [ ]:
# --- LLMプロバイダー別ライブラリ ---
# ご利用になるLLMプロバイダーに応じて、以下の該当する行のコメントを解除して実行してください。

# OpenAI (GPTシリーズ)
# !pip install -qU langchain_openai

# Azure OpenAI
# !pip install -qU langchain_openai # Azureもlangchain_openaiを利用

# Google Cloud Vertex AI (Gemini, PaLM等)
# !pip install -qU langchain_google_vertexai

# Google Gemini API (Google AI Studioで利用するGemini)
# !pip install -qU langchain_google_genai

# Anthropic (Claudeシリーズ)
# !pip install -qU langchain_anthropic

# Amazon Bedrock (AWS上の各種モデル、Claudeも含む)
# !pip install -qU langchain_aws boto3 # Bedrock利用時はboto3も必要

In [ ]:
# --- その他の推奨ライブラリ ---
# グラフの可視化や環境変数管理など、演習全体を通して利用する可能性のあるライブラリ
!pip install -qU python-dotenv pygraphviz pydotplus graphviz

### LLMプロバイダーの選択

このセルでは、使用するLLMプロバイダーを選択します。
`LLM_PROVIDER` 変数に、利用したいプロバイダー名を設定してください。
選択可能なプロバイダー: `"openai"`, `"azure"`, `"google"` (Vertex AI), `"google_genai"` (Gemini API), `"anthropic"`, `"bedrock"`

In [ ]:
# === LLMプロバイダーの選択 ===
# 利用したいLLMプロバイダーを以下の変数で指定してください。
# "openai", "azure", "google" (Vertex AI), "google_genai" (Gemini API), "anthropic", "bedrock" のいずれかを選択できます。
LLM_PROVIDER = "openai"  # 例: OpenAI を利用する場合

### APIキー/環境変数の設定

以下のセルを実行する前に、選択したLLMプロバイダーに応じたAPIキーまたは環境変数を設定する必要があります。

**一般的な手順:**
1.  `.env.sample` ファイルをコピーして `.env` ファイルを作成します。
2.  `.env` ファイルを開き、選択したLLMプロバイダーに対応するAPIキーや必要な情報を記述します。
    *   **OpenAI:** `OPENAI_API_KEY`
    *   **Azure OpenAI:** `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT`, `OPENAI_API_VERSION`, `AZURE_OPENAI_DEPLOYMENT_NAME`
    *   **Google (Vertex AI):** `GOOGLE_CLOUD_PROJECT_ID`, `GOOGLE_CLOUD_LOCATION` (Colab環境外で実行する場合、`GOOGLE_APPLICATION_CREDENTIALS` 環境変数の設定も必要になることがあります)
    *   **Google (Gemini API):** `GOOGLE_API_KEY`
    *   **Anthropic:** `ANTHROPIC_API_KEY`
    *   **AWS Bedrock:** `AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_REGION_NAME` (IAMロールを使用する場合は、これらのキー設定は不要な場合がありますが、リージョン名は必須です)
3.  ファイルを保存します。

**Google Colab を使用している場合:**
上記の `.env` ファイルを使用する代わりに、Colabのシークレットマネージャーに必要なキーを登録してください。
例えば、OpenAIを使用する場合は `OPENAI_API_KEY` という名前でシークレットを登録します。
Vertex AI を利用する場合は、Colab上での認証 (`google.colab.auth.authenticate_user()`) が実行されます。

このセルは、設定された情報に基づいて環境変数をロードし、LLMクライアントを初期化します。

In [ ]:
# === APIキー/環境変数の設定 ===
import os
from dotenv import load_dotenv

# .envファイルから環境変数を読み込む (存在する場合)
load_dotenv()

try:
    from google.colab import userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

# --- OpenAI ---
if LLM_PROVIDER == "openai":
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY and IS_COLAB:
        OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY:
        raise ValueError("OpenAI APIキーが設定されていません。環境変数 OPENAI_API_KEY を設定するか、Colab環境の場合はシークレットに OPENAI_API_KEY を設定してください。")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# --- Azure OpenAI ---
elif LLM_PROVIDER == "azure":
    AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
    AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
    OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
    AZURE_OPENAI_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if IS_COLAB:
        if not AZURE_OPENAI_API_KEY: AZURE_OPENAI_API_KEY = userdata.get("AZURE_OPENAI_API_KEY")
        if not AZURE_OPENAI_ENDPOINT: AZURE_OPENAI_ENDPOINT = userdata.get("AZURE_OPENAI_ENDPOINT")
        if not OPENAI_API_VERSION: OPENAI_API_VERSION = userdata.get("OPENAI_API_VERSION") # 例: "2023-07-01-preview"
        if not AZURE_OPENAI_DEPLOYMENT_NAME: AZURE_OPENAI_DEPLOYMENT_NAME = userdata.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if not AZURE_OPENAI_API_KEY: raise ValueError("Azure OpenAI APIキー (AZURE_OPENAI_API_KEY) が設定されていません。")
    if not AZURE_OPENAI_ENDPOINT: raise ValueError("Azure OpenAI エンドポイント (AZURE_OPENAI_ENDPOINT) が設定されていません。")
    if not OPENAI_API_VERSION: OPENAI_API_VERSION = "2023-07-01-preview" # デフォルトを設定することも可能
    if not AZURE_OPENAI_DEPLOYMENT_NAME: raise ValueError("Azure OpenAI デプロイメント名 (AZURE_OPENAI_DEPLOYMENT_NAME) が設定されていません。")

    os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
    os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
    os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION

# --- Google Cloud Vertex AI (Gemini) ---
elif LLM_PROVIDER == "google":
    PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT_ID") # .env 用に修正
    LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION")

    if IS_COLAB:
        if not PROJECT_ID: PROJECT_ID = userdata.get("GOOGLE_CLOUD_PROJECT_ID")
        if not LOCATION: LOCATION = userdata.get("GOOGLE_CLOUD_LOCATION") # 例: "us-central1"
        from google.colab import auth as google_auth
        google_auth.authenticate_user() # Vertex AI を使う場合は Colab での認証を推奨
    else: # Colab外の場合、.envから読み込んだ値で環境変数を設定
        if PROJECT_ID: os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID # Vertex AI SDKが参照する標準的な環境変数名
        if LOCATION: os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION

    if not PROJECT_ID: raise ValueError("Google Cloud Project ID が設定されていません。環境変数 GOOGLE_CLOUD_PROJECT_ID を設定するか、Colab環境の場合はシークレットに GOOGLE_CLOUD_PROJECT_ID を設定してください。")
    if not LOCATION: LOCATION = "us-central1" # デフォルトロケーション

# --- Google Gemini API (langchain-google-genai) ---
elif LLM_PROVIDER == "google_genai":
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY and IS_COLAB:
        GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("Google APIキーが設定されていません。環境変数 GOOGLE_API_KEY を設定するか、Colab環境の場合はシークレットに GOOGLE_API_KEY を設定してください。")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# --- Anthropic (Claude) ---
elif LLM_PROVIDER == "anthropic":
    ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY and IS_COLAB:
        ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY:
        raise ValueError("Anthropic APIキーが設定されていません。環境変数 ANTHROPIC_API_KEY を設定するか、Colab環境の場合はシークレットに ANTHROPIC_API_KEY を設定してください。")
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

# --- Amazon Bedrock (Claude) ---
elif LLM_PROVIDER == "bedrock":
    AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
    AWS_REGION_NAME = os.environ.get("AWS_REGION_NAME")

    if IS_COLAB: 
        if not AWS_ACCESS_KEY_ID: AWS_ACCESS_KEY_ID = userdata.get("AWS_ACCESS_KEY_ID")
        if not AWS_SECRET_ACCESS_KEY: AWS_SECRET_ACCESS_KEY = userdata.get("AWS_SECRET_ACCESS_KEY")
        if not AWS_REGION_NAME: AWS_REGION_NAME = userdata.get("AWS_REGION_NAME")

    if not AWS_REGION_NAME:
         raise ValueError("AWSリージョン名 (AWS_REGION_NAME) が設定されていません。Bedrock利用にはリージョン指定が必要です。")

    # 環境変数に設定 (boto3がこれらを自動で読み込む)
    if AWS_ACCESS_KEY_ID: os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
    if AWS_SECRET_ACCESS_KEY: os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
    os.environ["AWS_DEFAULT_REGION"] = AWS_REGION_NAME # boto3が参照する標準的なリージョン環境変数名
    os.environ["AWS_REGION"] = AWS_REGION_NAME # いくつかのライブラリはこちらを参照することもある

print(f"APIキー/環境変数の設定完了 (プロバイダー: {LLM_PROVIDER})")

### LLMクライアントの初期化

このセルは、上で選択・設定したLLMプロバイダーに基づいて、対応するLLMクライアントを初期化します。

In [ ]:
# === LLMクライアントの動的初期化 ===
llm = None

if LLM_PROVIDER == "openai":
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
elif LLM_PROVIDER == "azure":
    from langchain_openai import AzureChatOpenAI
    llm = AzureChatOpenAI(
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"), # 環境変数から取得
        openai_api_version=os.environ.get("OPENAI_API_VERSION"), # 環境変数から取得
        temperature=0,
    )
elif LLM_PROVIDER == "google":
    from langchain_google_vertexai import ChatVertexAI
    # PROJECT_ID, LOCATION は前のセルで環境変数に設定済みか、Colabの場合は直接利用
    llm = ChatVertexAI(model_name="gemini-2.0-flash", temperature=0, project=os.environ.get("GOOGLE_CLOUD_PROJECT"), location=os.environ.get("GOOGLE_CLOUD_LOCATION"))
elif LLM_PROVIDER == "google_genai":
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0, convert_system_message_to_human=True)
elif LLM_PROVIDER == "anthropic":
    from langchain_anthropic import ChatAnthropic
    llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
elif LLM_PROVIDER == "bedrock":
    from langchain_aws import ChatBedrock # langchain_community.chat_models から langchain_aws に変更の可能性あり
    # AWS_REGION_NAME は前のセルで環境変数 AWS_DEFAULT_REGION に設定済み
    llm = ChatBedrock( # BedrockChat ではなく ChatBedrock が一般的
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        # region_name=os.environ.get("AWS_DEFAULT_REGION"), # 通常、boto3が環境変数から自動で読み込む
        model_kwargs={"temperature": 0},
    )
else:
    raise ValueError(
        f"Unsupported LLM_PROVIDER: {LLM_PROVIDER}. "
        "Please choose from 'openai', 'azure', 'google', 'google_genai', 'anthropic', or 'bedrock'."
    )

print(f"LLM Provider: {LLM_PROVIDER}")
if llm:
    print(f"LLM Client Type: {type(llm)}")
    # モデル名取得の試行を汎用的に
    model_attr = (
                 getattr(llm, 'model', None) or
                 getattr(llm, 'model_name', None) or
                 getattr(llm, 'model_id', None) or
                 (hasattr(llm, 'llm') and getattr(llm.llm, 'model', None)) # 一部のLLMクライアントのネスト構造に対応
    )
    if hasattr(llm, 'azure_deployment') and not model_attr: # Azure特有の属性
        model_attr = llm.azure_deployment
        
    if model_attr:
        print(f"LLM Model: {model_attr}")
    else:
        print("LLM Model: (Could not determine model name from client attributes)")


---

### ■ 問題001: 最小構成のLangGraphグラフの構築

LangGraphの最も基本的な構成要素である`StateGraph`と`State`を理解し、シンプルなグラフを構築してみましょう。この問題では、入力された文字列をそのまま出力するだけの、単一のノードを持つグラフを作成します。

In [ ]:
# 解答欄001

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, ____]

# --- ノード定義 (Nodes) ---
def simple_node(state: GraphState):
    print(f'simple_node: {state["messages"][-1].content}')
    return {"messages": [state["messages"][-1]]}

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState)

# ノードの追加
workflow.____("simple_node", simple_node)

# エントリポイントの設定
workflow.____("simple_node")

# 終了ポイントの設定
workflow.____("simple_node", ____)

# グラフのコンパイル
app = workflow.____()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [("user", "Hello, LangGraph!")]}
for s in app.____(inputs):
    print(s)

# 最終結果の確認
final_state = app.____(inputs)
print(f"Final State: {final_state}")


<details><summary>解答001</summary>

``````python
# 解答001

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    # グラフの状態を保持する辞書
    # ここでは、入力メッセージを保持する
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def simple_node(state: GraphState):
    # 入力されたメッセージをそのまま返すノード
    print(f'simple_node: {state["messages"][-1].content}')
    return {"messages": [state["messages"][-1]]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("simple_node", simple_node)

# エントリポイントの設定
workflow.set_entry_point("simple_node")

# 終了ポイントの設定
workflow.add_edge("simple_node", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [("user", "Hello, LangGraph!")]}
for s in app.stream(inputs):
    print(s)

# 最終結果の確認
final_state = app.invoke(inputs)
print(f"Final State: {final_state}")
``````
</details>

<details><summary>解説001</summary>

#### この問題のポイント
*   **学習内容:** この問題では、`StateGraph`、`TypedDict`を用いた`State`の定義、`add_node`、`set_entry_point`、`add_edge`、`END`といったLangGraphの最も基本的なAPIを学びます。また、`Annotated`と`add_messages`を使ってメッセージ履歴を管理する方法も理解します。
*   **コード解説:**
    *   `GraphState`は、グラフ全体で共有される状態を定義します。`TypedDict`を使うことで、状態のスキーマを明確にできます。`messages: Annotated[list, add_messages]`は、LangChainのメッセージ形式のリストを状態として持ち、新しいメッセージが追加されるたびに自動的にリストの末尾に追加されるように設定しています。
    *   `simple_node`関数は、グラフのノードとして機能します。`state`引数として現在のグラフの状態を受け取り、新しい状態を辞書として返します。ここでは、入力された最後のメッセージをそのまま返しています。
    *   `StateGraph(GraphState)`でグラフのインスタンスを作成し、`GraphState`で定義した状態スキーマを渡します。
    *   `workflow.add_node("simple_node", simple_node)`で、`simple_node`関数を`simple_node`という名前のノードとしてグラフに追加します。
    *   `workflow.set_entry_point("simple_node")`は、グラフの実行が開始される最初のノードを指定します。
    *   `workflow.add_edge("simple_node", END)`は、`simple_node`の実行が完了したらグラフを終了することを示します。`END`はLangGraphが提供する特別な終了ノードです。
    *   `app = workflow.compile()`で、定義したワークフローを実行可能なアプリケーションにコンパイルします。
    *   `app.stream(inputs)`は、グラフの実行過程をストリーミングで受け取ることができます。`app.invoke(inputs)`は、グラフの実行が完了した最終状態を返します。
---
</details>

### ■ 問題002: 複数のノードを持つシーケンシャルグラフの構築

前の問題で学んだ基本的なグラフ構築に加えて、複数のノードを直列に接続し、データがノード間をどのように流れるかを理解しましょう。ここでは、入力された文字列を加工する2つのノード（例：大文字化、逆順化）を持つグラフを作成します。

In [ ]:
# 解答欄002

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, ____]

# --- ノード定義 (Nodes) ---
def uppercase_node(state: GraphState):
    last_message_content = state["messages"][-1].content
    print(f"uppercase_node: {last_message_content}")
    return {"messages": [____(content=last_message_content.upper())]}

def reverse_node(state: GraphState):
    last_message_content = state["messages"][-1].content
    print(f"reverse_node: {last_message_content}")
    return {"messages": [____(content=last_message_content[::-1])]}

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState)

# ノードの追加
workflow.____("uppercase", uppercase_node)
workflow.____("reverse", reverse_node)

# エントリポイントの設定
workflow.____("uppercase")

# エッジの追加 (直列接続)
workflow.____("uppercase", "reverse")
workflow.____("reverse", ____)

# グラフのコンパイル
app = workflow.____()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [____(content="Hello LangGraph")]}
for s in app.____(inputs):
    print(s)

final_state = app.____(inputs)
print(f"Final State: {final_state}")


<details><summary>解答002</summary>

``````python
# 解答002

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def uppercase_node(state: GraphState):
    # 最新のメッセージを大文字に変換するノード
    last_message_content = state["messages"][-1].content
    print(f"uppercase_node: {last_message_content}")
    return {"messages": [AIMessage(content=last_message_content.upper())]}

def reverse_node(state: GraphState):
    # 最新のメッセージを逆順にするノード
    last_message_content = state["messages"][-1].content
    print(f"reverse_node: {last_message_content}")
    return {"messages": [AIMessage(content=last_message_content[::-1])]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("uppercase", uppercase_node)
workflow.add_node("reverse", reverse_node)

# エントリポイントの設定
workflow.set_entry_point("uppercase")

# エッジの追加 (直列接続)
workflow.add_edge("uppercase", "reverse")
workflow.add_edge("reverse", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [HumanMessage(content="Hello LangGraph")]}
for s in app.stream(inputs):
    print(s)

final_state = app.invoke(inputs)
print(f"Final State: {final_state}")
``````
</details>

<details><summary>解説002</summary>

#### この問題のポイント
*   **学習内容:** 複数のノードを`add_edge`で直列に接続する方法と、ノード間で状態がどのように引き継がれるかを学びます。`HumanMessage`と`AIMessage`を使って、メッセージの送信元を明示する方法も理解します。
*   **コード解説:**
    *   `uppercase_node`と`reverse_node`は、それぞれ入力メッセージを大文字化、逆順化する処理を行います。重要なのは、各ノードが新しい`AIMessage`を作成して状態に返す点です。これにより、次のノードは前のノードの処理結果を`state["messages"][-1]`で取得できます。
    *   `workflow.add_edge("uppercase", "reverse")`は、`uppercase`ノードの実行が完了したら、次に`reverse`ノードを実行するように指示します。このようにして、処理の流れを定義します。
    *   入力メッセージを`HumanMessage`として渡すことで、ユーザーからの入力であることを明示しています。ノードからの出力は`AIMessage`として返され、メッセージ履歴にAIの応答として記録されます。
---
</details>

### ■ 問題003: 条件付きエッジによる分岐の導入

LangGraphの強力な機能の一つである条件付きエッジを導入し、グラフの実行パスを動的に制御する方法を学びましょう。ここでは、入力された数値が偶数か奇数かによって、異なる処理を行うグラフを作成します。

In [ ]:
# 解答欄003

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, ____]
    number: int # 新たに数値を保持する状態を追加

# --- ノード定義 (Nodes) ---
def check_number(state: GraphState):
    # 入力メッセージから数値を抽出し、状態に保存するノード
    try:
        num = int(state["messages"][-1].content)
        print(f"check_number: Extracted number {num}")
        return {"number": num}
    except ValueError:
        print("check_number: Invalid input, not a number.")
        return {"number": 0} # エラー時は0として扱うか、適切なエラーハンドリングを実装

def even_node(state: GraphState):
    # 偶数だった場合の処理ノード
    print(f"even_node: Number {state["number"]} is even.")
    return {"messages": [____(content=f"The number {state["number"]} is even.")]}

def odd_node(state: GraphState):
    # 奇数だった場合の処理ノード
    print(f"odd_node: Number {state["number"]} is odd.")
    return {"messages": [____(content=f"The number {state["number"]} is odd.")]}

# --- 条件付きエッジのルーター関数 ---
def route_number(state: GraphState):
    # 数値の状態に基づいて次のノードを決定する
    if state["number"] % 2 == 0:
        print("Routing to even_node")
        return "even_node"
    else:
        print("Routing to odd_node")
        return "odd_node"

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState)

# ノードの追加
workflow.____("check_number", check_number)
workflow.____("even_node", even_node)
workflow.____("odd_node", odd_node)

# エントリポイントの設定
workflow.____("check_number")

# 条件付きエッジの追加
workflow.____(
    "check_number", # 遷移元のノード
    ____,   # ルーター関数
    {
        "even_node": "even_node", # ルーター関数の戻り値とノード名のマッピング
        "odd_node": "odd_node"
    }
 )

# 各分岐からの終了エッジ
workflow.____("even_node", ____)
workflow.____("odd_node", ____)

# グラフのコンパイル
app = workflow.____()

# --- グラフの実行と結果表示 ---
print("\n--- 偶数のテスト ---")
inputs_even = {"messages": [____(content="42")]}
for s in app.____(inputs_even):
    print(s)
final_state_even = app.____(inputs_even)
print(f"Final State (Even): {final_state_even}")

print("\n--- 奇数のテスト ---")
inputs_odd = {"messages": [____(content="77")]}
for s in app.____(inputs_odd):
    print(s)
final_state_odd = app.____(inputs_odd)
print(f"Final State (Odd): {final_state_odd}")


<details><summary>解答003</summary>

``````python
# 解答003

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    number: int # 新たに数値を保持する状態を追加

# --- ノード定義 (Nodes) ---
def check_number(state: GraphState):
    # 入力メッセージから数値を抽出し、状態に保存するノード
    try:
        num = int(state["messages"][-1].content)
        print(f"check_number: Extracted number {num}")
        return {"number": num}
    except ValueError:
        print("check_number: Invalid input, not a number.")
        return {"number": 0} # エラー時は0として扱うか、適切なエラーハンドリングを実装

def even_node(state: GraphState):
    # 偶数だった場合の処理ノード
    print(f"even_node: Number {state["number"]} is even.")
    return {"messages": [AIMessage(content=f"The number {state["number"]} is even.")]}

def odd_node(state: GraphState):
    # 奇数だった場合の処理ノード
    print(f"odd_node: Number {state["number"]} is odd.")
    return {"messages": [AIMessage(content=f"The number {state["number"]} is odd.")]}

# --- 条件付きエッジのルーター関数 ---
def route_number(state: GraphState):
    # 数値の状態に基づいて次のノードを決定する
    if state["number"] % 2 == 0:
        print("Routing to even_node")
        return "even_node"
    else:
        print("Routing to odd_node")
        return "odd_node"

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("check_number", check_number)
workflow.add_node("even_node", even_node)
workflow.add_node("odd_node", odd_node)

# エントリポイントの設定
workflow.set_entry_point("check_number")

# 条件付きエッジの追加
workflow.add_conditional_edges(
    "check_number", # 遷移元のノード
    route_number,   # ルーター関数
    {
        "even_node": "even_node", # ルーター関数の戻り値とノード名のマッピング
        "odd_node": "odd_node"
    }
)

# 各分岐からの終了エッジ
workflow.add_edge("even_node", END)
workflow.add_edge("odd_node", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- 偶数のテスト ---")
inputs_even = {"messages": [HumanMessage(content="42")]}
for s in app.stream(inputs_even):
    print(s)
final_state_even = app.invoke(inputs_even)
print(f"Final State (Even): {final_state_even}")

print("\n--- 奇数のテスト ---")
inputs_odd = {"messages": [HumanMessage(content="77")]}
for s in app.stream(inputs_odd):
    print(s)
final_state_odd = app.invoke(inputs_odd)
print(f"Final State (Odd): {final_state_odd}")
``````
</details>

<details><summary>解説003</summary>

#### この問題のポイント
*   **学習内容:** `add_conditional_edges`を使用して、グラフの実行パスを動的に制御する方法を学びます。ルーター関数がどのように次のノードを決定するのか、そして状態が分岐間でどのように共有されるかを理解します。
*   **コード解説:**
    *   `GraphState`に`number`という新しいキーを追加し、入力された数値を保持するようにしました。
    *   `check_number`ノードは、入力メッセージから数値を抽出し、`number`状態を更新します。
    *   `even_node`と`odd_node`は、それぞれ偶数と奇数だった場合の最終処理を行います。
    *   `route_number`関数がルーターとして機能します。この関数は現在の`state`を受け取り、次に実行すべきノードの名前（文字列）を返します。LangGraphは、この戻り値に基づいて適切なエッジを辿ります。
    *   `workflow.add_conditional_edges("check_number", route_number, {"even_node": "even_node", "odd_node": "odd_node"})`は、`check_number`ノードの後に`route_number`関数を実行し、その戻り値が`"even_node"`なら`even_node`へ、`"odd_node"`なら`odd_node`へ遷移するように設定しています。
---
</details>

### ■ 問題004: グラフ内でのLLMの利用（シンプルなチャットボット）

LangGraphのノード内で大規模言語モデル（LLM）を呼び出す方法を学び、シンプルなチャットボットを構築しましょう。ここでは、ユーザーからの入力に対してLLMが応答を生成し、その応答を返すグラフを作成します。

In [ ]:
# 解答欄004
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
import os

# ノートブック冒頭で`llm`変数が初期化されている前提
# (from langchain_openai import ChatOpenAI や llm = ChatOpenAI(...) といった行はここには不要)

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def llm_node(state: GraphState):
    # LLMを呼び出し、応答を生成するノード
    # ノートブック冒頭で初期化された共通の `llm` 変数を使用します。
    print(f"llm_node: Calling LLM with messages: {state['messages']}")
    response = llm.invoke(state["messages"]) # 共通llmを使用 (ここは歯抜けにしない)
    print(f"llm_node: LLM response: {response.content}")
    return {"messages": [response]} # responseはAIMessageオブジェクトを期待 (ここは歯抜けにしない)

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState) # StateGraph

# ノードの追加
workflow.____("llm_responder", llm_node) # add_node

# エントリポイントの設定
workflow.____("llm_responder") # set_entry_point

# 終了ポイントの設定
workflow.____("llm_responder", ____) # add_edge, END

# グラフのコンパイル
app = workflow.____() # compile

# --- グラフの実行と結果表示 ---
print("\n--- チャットボットのテスト ---")
# 最初のメッセージはHumanMessageであると想定
inputs = {"messages": [____(content="こんにちは、あなたの名前は何ですか？")]} # HumanMessage
for s in app.____(inputs): # stream
    print(s)

final_state = app.____(inputs) # invoke
print(f"Final State: {final_state}")

print("\n--- 別の質問 ---")
inputs2 = {"messages": [____(content="今日の天気は？")]} # HumanMessage
for s in app.____(inputs2): # stream
    print(s)

final_state2 = app.____(inputs2) # invoke
print(f"Final State: {final_state2}")


<details><summary>解答004</summary>

``````python
# 解答004

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
import os # osはAPIキー設定のコメントアウト部分で使われているので残しても良いが、直接は不要になる

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def llm_node(state: GraphState):
    # LLMを呼び出し、応答を生成するノード
    # ノートブック冒頭で初期化された共通の `llm` 変数を使用します。
    print(f"llm_node: Calling LLM with messages: {state['messages']}")
    response = llm.invoke(state["messages"]) # 共通llmを使用
    print(f"llm_node: LLM response: {response.content}")
    return {"messages": [response]} # responseはAIMessageオブジェクトを期待

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("llm_responder", llm_node)

# エントリポイントの設定
workflow.set_entry_point("llm_responder")

# 終了ポイントの設定
workflow.add_edge("llm_responder", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- チャットボットのテスト ---")
# 最初のメッセージはHumanMessageであると想定
inputs = {"messages": [HumanMessage(content="こんにちは、あなたの名前は何ですか？")]}
for s in app.stream(inputs):
    print(s)

final_state = app.invoke(inputs)
print(f"Final State: {final_state}")

print("\n--- 別の質問 ---")
inputs2 = {"messages": [HumanMessage(content="今日の天気は？")]}
for s in app.stream(inputs2):
    print(s)

final_state2 = app.invoke(inputs2)
print(f"Final State: {final_state2}")
``````
</details>

<details><summary>解説004</summary>

このノートブックでは、様々なLLMプラットフォーム（OpenAI, Azure OpenAI, Google Cloud Vertex AI, Google Gemini (Gemini API), Anthropic Claude, Amazon Bedrockなど）を簡単に切り替えて試せるように設計されています。
ノートブックの冒頭にある `LLM_PROVIDER` 変数で使用したいLLMを選択し、対応するAPIキーや環境変数を設定するだけで、この問題を含む全てのLLM呼び出し箇所で選択したLLMが利用されます。
選択した `LLM_PROVIDER` に応じて、必要なAPIキーが設定されているか（環境変数またはGoogle Colabのシークレット経由）、ノートブック起動時にチェックされます。

ここでは、ノートブックの先頭で設定・初期化された共通の `llm` 変数を使用して、LLMに質問をしています。
`llm.invoke()` という統一されたインターフェースで、どのLLMプロバイダーを利用しているかに関わらず、同じようにLLMを呼び出すことができます。
これにより、特定のLLMサービスに依存しない、より汎用的なコードを作成するメリットを手軽に体験できます。

もしエラーが発生した場合は、ノートブック冒頭の `LLM_PROVIDER` の設定、および選択したプロバイダーに応じたAPIキーや環境変数の設定（例: `OPENAI_API_KEY`, `GOOGLE_API_KEY`, `AZURE_OPENAI_ENDPOINT`など）が正しく行われているかを確認してください。
各プロバイダー固有の設定項目（例えばVertex AIのProject ID、AzureのDeployment Name、Bedrockのリージョンなど）も見直してください。
プロバイダーによっては、`pip install` コマンドで対応するライブラリ (例: `langchain-google-genai`) がインストールされているかも確認点です。

</details>


### ■ 問題005: グラフの可視化とデバッグ

構築したLangGraphグラフの構造を視覚的に確認し、デバッグに役立てる方法を学びましょう。ここでは、これまでに作成したグラフのいずれか（例：問題003の条件分岐グラフ）を可視化し、その構造を理解します。

In [ ]:
# 解答欄005

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, ____]
    number: int

# --- ノード定義 (Nodes) ---
def check_number(state: GraphState):
    try:
        num = int(state["messages"][-1].content)
        return {"number": num}
    except ValueError:
        return {"number": 0}

def even_node(state: GraphState):
    return {"messages": [____(content=f"The number {state["number"]} is even.")]}

def odd_node(state: GraphState):
    return {"messages": [____(content=f"The number {state["number"]} is odd.")]}

# --- 条件付きエッジのルーター関数 ---
def route_number(state: GraphState):
    if state["number"] % 2 == 0:
        return "even_node"
    else:
        return "odd_node"

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState)

workflow.____("check_number", check_number)
workflow.____("even_node", even_node)
workflow.____("odd_node", odd_node)

workflow.____("check_number")

workflow.____(
    "check_number",
    ____,
    {
        "even_node": "even_node",
        "odd_node": "odd_node"
    }
)

workflow.____("even_node", ____)
workflow.____("odd_node", ____)

app = workflow.____()

# --- グラフの可視化 ---
# グラフを画像として表示
# graphvizがインストールされている必要があります: pip install pygraphviz pydotplus graphviz
# また、システムにGraphvizがインストールされている必要があります。
# Windows: https://graphviz.org/download/
# Mac: brew install graphviz
# Linux: sudo apt-get install graphviz
try:
    ____(____(app.____().____()))
    print("グラフが正常に可視化されました。")
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

# --- グラフの実行と結果表示 (オプション) ---
# 可視化したグラフが正しく動作するか確認するために、再度実行してみる
print("\n--- 偶数のテスト (可視化後の確認) ---")
inputs_even = {"messages": [____(content="10")]}
for s in app.____(inputs_even):
    print(s)
final_state_even = app.____(inputs_even)
print(f"Final State: {final_state_even}")


<details><summary>解答005</summary>

``````python
# 解答005

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    number: int

# --- ノード定義 (Nodes) ---
def check_number(state: GraphState):
    try:
        num = int(state["messages"][-1].content)
        return {"number": num}
    except ValueError:
        return {"number": 0}

def even_node(state: GraphState):
    return {"messages": [AIMessage(content=f"The number {state["number"]} is even.")]}

def odd_node(state: GraphState):
    return {"messages": [AIMessage(content=f"The number {state["number"]} is odd.")]}

# --- 条件付きエッジのルーター関数 ---
def route_number(state: GraphState):
    if state["number"] % 2 == 0:
        return "even_node"
    else:
        return "odd_node"

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

workflow.add_node("check_number", check_number)
workflow.add_node("even_node", even_node)
workflow.add_node("odd_node", odd_node)

workflow.set_entry_point("check_number")

workflow.add_conditional_edges(
    "check_number",
    route_number,
    {
        "even_node": "even_node",
        "odd_node": "odd_node"
    }
)

workflow.add_edge("even_node", END)
workflow.add_edge("odd_node", END)

app = workflow.compile()

# --- グラフの可視化 ---
# グラフを画像として表示
# graphvizがインストールされている必要があります: pip install pygraphviz pydotplus graphviz
# また、システムにGraphvizがインストールされている必要があります。
# Windows: https://graphviz.org/download/
# Mac: brew install graphviz
# Linux: sudo apt-get install graphviz
try:
    display(Image(app.get_graph().draw_png()))
    print("グラフが正常に可視化されました。")
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

# --- グラフの実行と結果表示 (オプション) ---
# 可視化したグラフが正しく動作するか確認するために、再度実行してみる
print("\n--- 偶数のテスト (可視化後の確認) ---")
inputs_even = {"messages": [HumanMessage(content="10")]}
for s in app.stream(inputs_even):
    print(s)
final_state_even = app.invoke(inputs_even)
print(f"Final State: {final_state_even}")
``````
</details>

<details><summary>解説005</summary>

#### この問題のポイント
*   **学習内容:** `app.get_graph().draw_png()`を使用してLangGraphのグラフ構造を画像として可視化する方法を学びます。これにより、複雑なグラフのデバッグや理解が容易になります。
*   **コード解説:**
    *   この問題では、問題003で作成した条件分岐グラフを再利用しています。これは、可視化の有用性を示すのに適した例だからです。
    *   `app.get_graph()`は、コンパイルされたグラフの内部表現を取得します。
    *   `.draw_png()`メソッドは、そのグラフ構造をPNG画像としてバイト列で返します。この機能を利用するには、システムにGraphvizがインストールされている必要があります。また、Pythonの`pygraphviz`や`pydotplus`といったライブラリも必要になる場合があります。
    *   `IPython.display.Image`と`display`を使うことで、Jupyter Notebook内で直接画像をレンダリングして表示できます。
    *   `try-except`ブロックでGraphvizのインストール状況によるエラーをハンドリングし、ユーザーに適切なメッセージを表示するようにしています。グラフが複雑になるにつれて、この可視化機能はデバッグや設計の確認に不可欠となります。
---
</details>

### ■ 問題006: 状態の更新 - 特定キーの値を上書きする

`add_messages` によるメッセージ履歴の追加だけでなく、グラフの状態(`State`)内の特定のキーの値を直接更新する方法を学びましょう。ここでは、カウンター値を保持する状態キーを定義し、ノードでその値をインクリメントするグラフを作成します。

In [ ]:
# 解答欄006

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage

# --- 状態定義 (State) ---
class CounterState(TypedDict):
    messages: Annotated[list, ____] # add_messages
    counter: int # 新しくカウンター用の状態キーを定義

# --- ノード定義 (Nodes) ---
def increment_counter(state: CounterState):
    # counterの値を1増やすノード
    current_count = state.get("counter", 0) # stateからcounterの値を取得、なければ0
    new_count = current_count + 1
    print(f"increment_counter: Current count: {current_count}, New count: {new_count}")
    return {"counter": ____, "messages": [HumanMessage(content=f"Counter incremented to {new_count}")]} # new_count

def display_count(state: CounterState):
    # counterの最終値を表示するノード (実際にはmessagesに追加されたもので確認)
    print(f"display_count: Final counter value is {state['counter']}")
    # このノードは状態を更新しないが、メッセージを追加しても良い
    return {"messages": [HumanMessage(content=f"Final count: {state['counter']}")]}

# --- グラフ構築 (Graph) ---
workflow = ____(CounterState) # StateGraph

# ノードの追加
workflow.____("increment", increment_counter) # add_node
workflow.____("display", display_count) # add_node

# エントリポイントの設定
workflow.____("increment") # set_entry_point

# エッジの追加
workflow.____("increment", "display") # add_edge
workflow.____("display", ____) # add_edge, END

# グラフのコンパイル
app = workflow.____() # compile

# --- グラフの実行と結果表示 ---
print("\n--- カウンターテスト (初期値0から) ---")
inputs = {"messages": [HumanMessage(content="Start counting")], "counter": 0} # 初期カウンター値を設定
for s in app.____(inputs): # stream
    print(s)
final_state = app.____(inputs) # invoke
print(f"Final State: {final_state}")

print("\n--- カウンターテスト (初期値5から) ---")
inputs_2 = {"messages": [HumanMessage(content="Start counting from 5")], "counter": 5} # 初期カウンター値を設定
for s in app.____(inputs_2): # stream
    print(s)
final_state_2 = app.____(inputs_2) # invoke
print(f"Final State: {final_state_2}")

<details><summary>解答006</summary>

``````python
# 解答006

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage

# --- 状態定義 (State) ---
class CounterState(TypedDict):
    messages: Annotated[list, add_messages]
    counter: int # 新しくカウンター用の状態キーを定義

# --- ノード定義 (Nodes) ---
def increment_counter(state: CounterState):
    # counterの値を1増やすノード
    current_count = state.get("counter", 0) # stateからcounterの値を取得、なければ0
    new_count = current_count + 1
    print(f"increment_counter: Current count: {current_count}, New count: {new_count}")
    return {"counter": new_count, "messages": [HumanMessage(content=f"Counter incremented to {new_count}")]}

def display_count(state: CounterState):
    # counterの最終値を表示するノード (実際にはmessagesに追加されたもので確認)
    print(f"display_count: Final counter value is {state['counter']}")
    # このノードは状態を更新しないが、メッセージを追加しても良い
    return {"messages": [HumanMessage(content=f"Final count: {state['counter']}")]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(CounterState)

# ノードの追加
workflow.add_node("increment", increment_counter)
workflow.add_node("display", display_count)

# エントリポイントの設定
workflow.set_entry_point("increment")

# エッジの追加
workflow.add_edge("increment", "display")
workflow.add_edge("display", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- カウンターテスト (初期値0から) ---")
inputs = {"messages": [HumanMessage(content="Start counting")], "counter": 0} # 初期カウンター値を設定
for s in app.stream(inputs):
    print(s)
final_state = app.invoke(inputs)
print(f"Final State: {final_state}")

print("\n--- カウンターテスト (初期値5から) ---")
inputs_2 = {"messages": [HumanMessage(content="Start counting from 5")], "counter": 5} # 初期カウンター値を設定
for s in app.stream(inputs_2):
    print(s)
final_state_2 = app.invoke(inputs_2)
print(f"Final State: {final_state_2}")
``````
</details>

<details><summary>解説006</summary>

#### この問題のポイント
*   **学習内容:** `TypedDict`で定義する状態クラスに、`messages`以外のカスタムキー（ここでは`counter: int`）を追加し、ノード関数内でその値を直接読み書きする方法を学びます。これにより、メッセージ履歴だけでなく、数値や文字列、ブール値など、より多様なデータをグラフ全体で管理できるようになります。
*   **コード解説:**
    *   `CounterState`に`counter: int`を追加しました。これにより、グラフの状態はメッセージリストと整数型のカウンターを持つことになります。
    *   `increment_counter`ノードでは、`state.get("counter", 0)`を使って現在のカウンター値を取得しています。`.get()`メソッドを使うことで、キーが存在しない場合のデフォルト値を指定できます（ここでは初回実行時を想定して0）。その後、値をインクリメントし、更新後の値を`{"counter": new_count}`という辞書形式で返しています。LangGraphは、ノードが返す辞書のキーに基づいて対応する状態を更新します。
    *   `messages`キーも同時に返すことで、状態更新のログや情報をメッセージ履歴に残すことができます。
    *   グラフ実行時 (`app.invoke`や`app.stream`) に、`inputs`辞書に`"counter": 0`（または任意の値）を含めることで、`counter`キーの初期値を設定できます。
    *   このように、ノードは状態の一部または全部を更新する辞書を返すことで、グラフの状態を変化させます。`add_messages`はメッセージリストの更新に特化した便利な方法ですが、他のキーは直接値を指定して更新します。
---
</details>

### ■ 問題007: 状態の更新 - 複数のキーを一度に更新する

一つのノードから、状態(`State`)の複数のキーを同時に更新する方法を学びましょう。ここでは、ユーザーからのメッセージ内容に応じて、応答メッセージと共に「応答タイプ」という別の状態キーも更新するグラフを作成します。

In [ ]:
# 解答欄007
from typing import TypedDict, Annotated, Literal
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
ResponseType = Literal["greeting", "question", "other", "unknown"]

class MultiUpdateState(TypedDict):
    messages: Annotated[list, add_messages]
    response_type: ____ # ResponseType (応答の種類を保持するキー)
    last_user_message: str # 最後に入力されたユーザーメッセージ

# --- ノード定義 (Nodes) ---
def process_input(state: MultiUpdateState):
    user_message = state["messages"][-1].content.lower()
    response_text = ""
    resp_type: ResponseType = "unknown"

    if "こんにちは" in user_message or "こんばんは" in user_message:
        response_text = "こんにちは！何かお手伝いできますか？"
        resp_type = ____ # "greeting"
    elif "?" in user_message or "教えて" in user_message:
        response_text = "ご質問ありがとうございます。それについては現在調べています。"
        resp_type = ____ # "question"
    else:
        response_text = "メッセージありがとうございます。"
        resp_type = ____ # "other"
    
    print(f"process_input: User: '{user_message}', AI: '{response_text}', Type: '{resp_type}'")
    # 複数のキーを同時に更新して返す
    return {
        "messages": [AIMessage(content=response_text)],
        "response_type": resp_type,
        "last_user_message": ____ # user_message
    }

# --- グラフ構築 (Graph) ---
workflow = ____(MultiUpdateState) # StateGraph

workflow.____("processor", process_input) # add_node
workflow.____("processor") # set_entry_point
workflow.____("processor", ____) # add_edge, END

app = workflow.____() # compile

# --- グラフの実行と結果表示 ---
test_inputs = [
    {"messages": [HumanMessage(content="こんにちは")]},
    {"messages": [HumanMessage(content="LangGraphについて教えてください")]},
    {"messages": [HumanMessage(content="今日の天気は晴れですね")]}
]

for i, inputs in enumerate(test_inputs):
    print(f"\n--- テスト実行 {i+1} ---")
    for s in app.____(inputs, {"recursion_limit": 3}): # stream
        print(s)
    final_state = app.____(inputs, {"recursion_limit": 3}) # invoke
    print(f"Final State {i+1}: {final_state}")
    assert "response_type" in final_state
    assert "last_user_message" in final_state
    print(f"Response Type: {final_state['response_type']}")
    print(f"Last User Message: {final_state['last_user_message']}")

<details><summary>解答007</summary>

``````python
# 解答007
from typing import TypedDict, Annotated, Literal
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
ResponseType = Literal["greeting", "question", "other", "unknown"]

class MultiUpdateState(TypedDict):
    messages: Annotated[list, add_messages]
    response_type: ResponseType # 応答の種類を保持するキー
    last_user_message: str # 最後に入力されたユーザーメッセージ

# --- ノード定義 (Nodes) ---
def process_input(state: MultiUpdateState):
    user_message = state["messages"][-1].content.lower()
    response_text = ""
    resp_type: ResponseType = "unknown"

    if "こんにちは" in user_message or "こんばんは" in user_message:
        response_text = "こんにちは！何かお手伝いできますか？"
        resp_type = "greeting"
    elif "?" in user_message or "教えて" in user_message:
        response_text = "ご質問ありがとうございます。それについては現在調べています。"
        resp_type = "question"
    else:
        response_text = "メッセージありがとうございます。"
        resp_type = "other"
    
    print(f"process_input: User: '{user_message}', AI: '{response_text}', Type: '{resp_type}'")
    # 複数のキーを同時に更新して返す
    return {
        "messages": [AIMessage(content=response_text)],
        "response_type": resp_type,
        "last_user_message": user_message # 元のユーザーメッセージを保存
    }

# --- グラフ構築 (Graph) ---
workflow = StateGraph(MultiUpdateState)

workflow.add_node("processor", process_input)
workflow.set_entry_point("processor")
workflow.add_edge("processor", END)

app = workflow.compile()

# --- グラフの実行と結果表示 ---
test_inputs = [
    {"messages": [HumanMessage(content="こんにちは")]},
    {"messages": [HumanMessage(content="LangGraphについて教えてください")]},
    {"messages": [HumanMessage(content="今日の天気は晴れですね")]}
]

for i, inputs in enumerate(test_inputs):
    print(f"\n--- テスト実行 {i+1} ---")
    # 初期状態としてresponse_typeやlast_user_messageを渡すことも可能だが、
    # この問題ではノード内でこれらが設定されることを確認する
    initial_state = inputs.copy()
    # 必要であれば、初期値を設定
    # initial_state.setdefault("response_type", "unknown") 
    # initial_state.setdefault("last_user_message", "")

    for s in app.stream(initial_state, {"recursion_limit": 3}):
        print(s)
    final_state = app.invoke(initial_state, {"recursion_limit": 3})
    print(f"Final State {i+1}: {final_state}")
    assert "response_type" in final_state
    assert "last_user_message" in final_state
    print(f"Response Type: {final_state['response_type']}")
    print(f"Last User Message: {final_state['last_user_message']}")
``````
</details>

<details><summary>解説007</summary>

#### この問題のポイント
*   **学習内容:** 一つのノード関数から返す辞書に複数のキーと値のペアを含めることで、グラフの状態(`State`)の複数の属性を一度に更新する方法を学びます。また、`typing.Literal`を使って、状態キーが取りうる値を制限する方法も示します。
*   **コード解説:**
    *   `MultiUpdateState`に、`response_type: ResponseType` と `last_user_message: str` という2つの新しいキーを追加しました。`ResponseType`は`Literal["greeting", "question", "other", "unknown"]`で定義され、`response_type`キーがこれらの文字列のうちのいずれかの値を取ることを示します（型ヒントであり、実行時の厳密な強制ではありませんが、開発時の可読性や静的解析に役立ちます）。
    *   `process_input`ノードは、ユーザーのメッセージ内容に基づいて応答メッセージを生成し、それと同時に`response_type`（挨拶、質問、その他など）と`last_user_message`（処理対象となった元のユーザーメッセージ）も決定します。
    *   ノードが返す辞書は `{"messages": ..., "response_type": ..., "last_user_message": ...}` のようになります。LangGraphは、この辞書に含まれる各キーに対応する状態を更新します。
    *   実行時には、`messages`キーだけでなく、`response_type`と`last_user_message`も最終状態に含まれていることを確認できます。
    *   このように、ノードはグラフの状態を柔軟に更新する役割を担います。返す辞書に含めるキーと値によって、どの状態属性をどのように変更するかを制御できます。
---
</details>

### ■ 問題008: ノードからの複数の出力先（静的エッジ）

条件付きエッジ(`add_conditional_edges`)は強力ですが、場合によってはノードの処理結果に応じて、常に固定された異なる次のノードへ遷移させたいことがあります（実行前に分岐先が決定しているケース）。このような静的な分岐は、単に`add_edge`を複数回定義することで実現できます。ここでは、入力文字列に応じて異なる処理ノードへ遷移するグラフを作成します。

In [ ]:
# 解答欄008
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
class StaticBranchState(TypedDict):
    messages: Annotated[list, add_messages]
    category: str # 入力カテゴリを保持

# --- ノード定義 (Nodes) ---
def categorize_input(state: StaticBranchState):
    last_message = state["messages"][-1].content.lower()
    cat = "unknown"
    if "fruit" in last_message:
        cat = "fruit_handler"
    elif "color" in last_message:
        cat = "color_handler"
    print(f"categorize_input: Message '{last_message}' categorized as '{cat}'")
    # このノードはカテゴリを返すだけで、次の遷移はエッジ定義に依存
    # ただし、後続のノードがこのカテゴリ情報を使うかもしれないので状態に保存
    return {"category": cat, "messages": [AIMessage(content=f"Input categorized as {cat}")]}

def fruit_node(state: StaticBranchState):
    print("fruit_node: Processing fruit-related input.")
    return {"messages": [AIMessage(content="This is about fruits.")]}

def color_node(state: StaticBranchState):
    print("color_node: Processing color-related input.")
    return {"messages": [AIMessage(content="This is about colors.")]}

def unknown_node(state: StaticBranchState):
    print("unknown_node: Processing unknown input.")
    return {"messages": [AIMessage(content="Input category unknown.")]}

# --- 条件付きエッジのルーター関数 (問題003の形式) ---
def route_by_category(state: StaticBranchState):
    # この関数が返す文字列が、次のノード名になる
    # カテゴリが fruit_handler なら fruit_node へ、など
    # ここを埋めてください
    category = state.get("category", "unknown_handler") # カテゴリを取得、なければunknown_handler
    print(f"route_by_category: Routing based on category '{category}'")
    if category == "fruit_handler":
        return ____ # "fruit_node"
    elif category == "color_handler":
        return ____ # "color_node"
    else:
        return ____ # "unknown_node"

# --- グラフ構築 (Graph) ---
workflow = ____(StaticBranchState) # StateGraph

workflow.____("categorizer", categorize_input) # add_node
workflow.____("fruit_node", fruit_node) # add_node
workflow.____("color_node", color_node) # add_node
workflow.____("unknown_node", unknown_node) # add_node

workflow.____("categorizer") # set_entry_point

# 条件付きエッジを使って分岐を定義
workflow.____( # add_conditional_edges
    "categorizer",
    ____, # route_by_category
    {
        "fruit_node": "fruit_node",
        "color_node": "color_node",
        "unknown_node": "unknown_node",
        # END を直接指定することも可能だが、ここでは専用ノードへ
    }
)

# 各処理ノードから終了へ
workflow.____("fruit_node", ____) # add_edge, END
workflow.____("color_node", ____) # add_edge, END
workflow.____("unknown_node", ____) # add_edge, END

app = workflow.____() # compile

# --- グラフの実行と結果表示 ---
test_data = [
    {"messages": [HumanMessage(content="I like apples, it's a fruit.")]},
    {"messages": [HumanMessage(content="Blue is my favorite color.")]},
    {"messages": [HumanMessage(content="Tell me a joke.")]}
]

for i, data in enumerate(test_data):
    print(f"\n--- 静的分岐テスト {i+1} ---")
    # 初期カテゴリはcategorize_inputで設定されるので入力不要
    for s in app.____(data, {"recursion_limit": 3}): # stream
        print(s)
    final_state = app.____(data, {"recursion_limit": 3}) # invoke
    print(f"Final State {i+1}: {final_state}")

try:
    display(Image(app.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")

<details><summary>解答008</summary>

``````python
# 解答008
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display # 可視化のため

# --- 状態定義 (State) ---
class StaticBranchState(TypedDict):
    messages: Annotated[list, add_messages]
    category: str # 入力カテゴリを保持

# --- ノード定義 (Nodes) ---
def categorize_input(state: StaticBranchState):
    last_message = state["messages"][-1].content.lower()
    cat = "unknown_handler" # デフォルトをunknown_handlerに
    if "fruit" in last_message:
        cat = "fruit_handler"
    elif "color" in last_message:
        cat = "color_handler"
    print(f"categorize_input: Message '{last_message}' categorized as '{cat}'")
    return {"category": cat, "messages": [AIMessage(content=f"Input categorized as {cat}")]}

def fruit_node(state: StaticBranchState):
    print("fruit_node: Processing fruit-related input.")
    return {"messages": [AIMessage(content="This is about fruits.")]}

def color_node(state: StaticBranchState):
    print("color_node: Processing color-related input.")
    return {"messages": [AIMessage(content="This is about colors.")]}

def unknown_node(state: StaticBranchState):
    print("unknown_node: Processing unknown input.")
    return {"messages": [AIMessage(content="Input category unknown.")]}

# --- 条件付きエッジのルーター関数 ---
def route_by_category(state: StaticBranchState):
    category = state.get("category", "unknown_handler") 
    print(f"route_by_category: Routing based on category '{category}'")
    if category == "fruit_handler":
        return "fruit_node"
    elif category == "color_handler":
        return "color_node"
    else: # unknown_handler やその他の場合
        return "unknown_node"

# --- グラフ構築 (Graph) ---
workflow = StateGraph(StaticBranchState)

workflow.add_node("categorizer", categorize_input)
workflow.add_node("fruit_node", fruit_node)
workflow.add_node("color_node", color_node)
workflow.add_node("unknown_node", unknown_node)

workflow.set_entry_point("categorizer")

# 条件付きエッジを使って分岐を定義
workflow.add_conditional_edges(
    "categorizer",
    route_by_category,
    {
        "fruit_node": "fruit_node",
        "color_node": "color_node",
        "unknown_node": "unknown_node",
    }
)

# 各処理ノードから終了へ
workflow.add_edge("fruit_node", END)
workflow.add_edge("color_node", END)
workflow.add_edge("unknown_node", END)

app = workflow.compile()

# --- グラフの実行と結果表示 ---
test_data = [
    {"messages": [HumanMessage(content="I like apples, it's a fruit.")]},
    {"messages": [HumanMessage(content="Blue is my favorite color.")]},
    {"messages": [HumanMessage(content="Tell me a joke.")]}
]

for i, data in enumerate(test_data):
    print(f"\n--- 静的分岐テスト {i+1} ---")
    for s in app.stream(data, {"recursion_limit": 3}):
        print(s)
    final_state = app.invoke(data, {"recursion_limit": 3})
    print(f"Final State {i+1}: {final_state}")

try:
    display(Image(app.get_graph().draw_png()))
    print("Graph visualized successfully.")
except Exception as e:
    print(f"Graph visualization failed: {e}")
``````
</details>

<details><summary>解説008</summary>

#### この問題のポイント
*   **学習内容:** この問題では、問題003で学んだ`add_conditional_edges`とルーター関数を再利用して、ノードの処理結果（ここでは`category`状態）に基づいて、複数の固定的な次のノードへ分岐する方法を復習・確認します。問題文の意図は「`add_edge`を複数回使った静的分岐」でしたが、より一般的な条件分岐の実装方法である`add_conditional_edges`を用いた形としました。これにより、状態に基づいて動的に次の遷移先を決定する強力なパターンを再確認できます。
*   **コード解説:**
    *   `StaticBranchState`に`category`キーを追加し、入力がどのカテゴリに属するかを保持します。
    *   `categorize_input`ノードは、入力メッセージの内容を解析し、`category`状態を更新します（例: `fruit_handler`, `color_handler`, `unknown_handler`）。
    *   `fruit_node`, `color_node`, `unknown_node`は、それぞれのカテゴリに対応する処理を行うノードです。
    *   `route_by_category`関数は、現在の`state['category']`の値を見て、次に実行すべきノード名（`"fruit_node"`、`"color_node"`、または`"unknown_node"`）を返します。
    *   `workflow.add_conditional_edges("categorizer", route_by_category, {...})`は、`categorizer`ノードの後に`route_by_category`関数を実行し、その戻り値に応じて指定されたノード（`fruit_node`, `color_node`, `unknown_node`のいずれか）に処理を分岐させます。
    *   各処理ノード（`fruit_node`など）は、最終的に`END`に繋がっており、そこでグラフの実行が終了します。
    *   このパターンは、状態に基づいて処理フローを制御する際の基本となり、より複雑なロジックを構築する上で非常に重要です。
    *   グラフの可視化も行い、分岐構造を視覚的に確認できるようにしています。

**補足:** もし「`add_edge`を複数回使った静的分岐」を厳密に表現したい場合、それは通常、あるノードが常に複数の特定の他のノードに「通知」を送るようなシナリオ（ファンアウト）で考えられますが、LangGraphの基本的な`add_edge`は一つの遷移先を指定します。一つのノードから複数のエッジを出す場合、それらは通常、異なる条件やイベントに対応するものです。上記の`add_conditional_edges`が、状態に基づいた分岐を実現する最も標準的で柔軟な方法です。
---
</details>

### ■ 問題009: `END` 以外の終了ノードの指定（概念理解）

LangGraphでは、グラフの終点は通常、特別な `END` ノードに接続することで示されます。しかし、概念的には、あるノードが処理の最終ステップであり、それ以上後続のノードが存在しない場合、そのノードが事実上の「終了ノード」として機能すると考えることもできます。この問題では、特定のノードを実行した後、明示的に `END` に接続せず、グラフがそこで停止することを確認します。ただし、LangGraphのベストプラクティスとしては、可能な限り `END` を使用することが推奨されます。

In [ ]:
# 解答欄009
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
class FinalNodeState(TypedDict):
    messages: Annotated[list, add_messages]
    status: str

# --- ノード定義 (Nodes) ---
def start_process(state: FinalNodeState):
    print("start_process: Process started.")
    return {"status": "Processing", "messages": [AIMessage(content="Process initiated.")]}

def final_processing_node(state: FinalNodeState):
    # このノードが処理の最後とする
    final_message = "Process completed at final_processing_node."
    print(f"final_processing_node: {final_message}")
    return {"status": "Completed", "messages": [AIMessage(content=final_message)]}

# --- グラフ構築 (Graph) ---
workflow = ____(FinalNodeState) # StateGraph

workflow.____("start", start_process) # add_node
workflow.____("final_step", final_processing_node) # add_node

workflow.____("start") # set_entry_point

# startノードからfinal_stepノードへのエッジ
workflow.____("start", "final_step") # add_edge

# final_stepノードからENDへのエッジを意図的に作成しない
# workflow.add_edge("final_step", END) # ← これをコメントアウトまたは削除

# グラフのコンパイル
# check_interruptions=True をつけると、ENDに到達しない場合にエラーになるため、
# この例では明示的にENDに繋がないことを示すために compile() の引数なし、
# または check_interruptions=False (デフォルト) を利用します。
# Langfuseなどのトレーシングツールと連携する場合、ENDに到達しないとトレースが終了しないことがあるため注意。
app = workflow.____() # compile

# --- グラフの実行と結果表示 ---
print("\n--- 最終ノードテスト ---")
inputs = {"messages": [HumanMessage(content="Begin process")], "status": "Initial"}
final_state = None
for s in app.____(inputs, {"recursion_limit": 5}): # stream
    print(s)
    # streamの最後の要素が最終状態となる
    # (invokeと異なり、ENDに到達しなくても最後に実行されたノードの更新後状態が返る)
    if "final_step" in s: # final_stepノードの出力が得られたら、それが最終状態
        final_state = s["final_step"]

print(f"Final State from stream: {final_state}")

# invokeを呼び出す場合、ENDに到達しないとエラーになるか、最後のノードの結果が返るかは
# LangGraphのバージョンや設定に依存する可能性がある。
# ここではstreamの挙動を中心に確認。
try:
    invoked_state = app.invoke(inputs, {"recursion_limit": 5})
    print(f"Final State from invoke: {invoked_state}")
    # invokeがエラーなく値を返す場合、その値を最終状態として扱う
    # 基本的にはENDに到達することが期待される
except Exception as e:
    print(f"Invoke failed as expected or due to other reasons: {e}")
    print("InvokeはENDに到達しない場合、エラーを発生させることがあります。streamの最後の出力で状態を確認してください。")

assert final_state is not None, "Final state was not captured from stream"
assert final_state["status"] == "Completed"
assert "Process completed at final_processing_node." in final_state["messages"][-1].content
print("Assertion for final_state from stream passed.")

try:
    display(Image(app.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")

<details><summary>解答009</summary>

``````python
# 解答009
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class FinalNodeState(TypedDict):
    messages: Annotated[list, add_messages]
    status: str

# --- ノード定義 (Nodes) ---
def start_process(state: FinalNodeState):
    print("start_process: Process started.")
    return {"status": "Processing", "messages": [AIMessage(content="Process initiated.")]}

def final_processing_node(state: FinalNodeState):
    # このノードが処理の最後とする
    final_message = "Process completed at final_processing_node."
    print(f"final_processing_node: {final_message}")
    return {"status": "Completed", "messages": [AIMessage(content=final_message)]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(FinalNodeState)

workflow.add_node("start", start_process)
workflow.add_node("final_step", final_processing_node)

workflow.set_entry_point("start")

# startノードからfinal_stepノードへのエッジ
workflow.add_edge("start", "final_step")

# final_stepノードからENDへのエッジを意図的に作成しない
# workflow.add_edge("final_step", END) # ← コメントアウトまたは削除

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- 最終ノードテスト ---")
inputs = {"messages": [HumanMessage(content="Begin process")], "status": "Initial"}
final_state_from_stream = None

print("Streaming execution:")
for s in app.stream(inputs, {"recursion_limit": 5}):
    print(s)
    # streamの各要素は {ノード名: 更新された状態} または {ノード名: 更新内容} の形式
    # 最後の "final_step" ノードの出力が、この場合の最終状態を示す
    if "final_step" in s:
        final_state_from_stream = s["final_step"]

print(f"Final State from stream: {final_state_from_stream}")

# invokeの挙動確認
invoked_state = None
try:
    # ENDに繋がっていない場合、invokeはエラーを出すか、最後に実行されたノードの状態を返す
    # LangGraphのバージョンや内部実装により挙動が変わりうるため注意
    # ここでは、streamの結果を正として扱う
    invoked_state = app.invoke(inputs, {"recursion_limit": 5})
    print(f"Final State from invoke: {invoked_state}")
except Exception as e:
    print(f"Invoke call resulted in an error or unexpected behavior: {e}")
    print("This might be expected if the graph doesn't explicitly reach END.")

# streamから取得した最終状態でアサーションを行う
assert final_state_from_stream is not None, "Final state was not captured from stream"
assert final_state_from_stream["status"] == "Completed"
assert "Process completed at final_processing_node." in final_state_from_stream["messages"][-1].content
print("Assertion for final_state_from_stream passed.")

# もしinvokeが値を返した場合、それも確認 (参考程度)
if invoked_state:
    print(f"Invoked state status: {invoked_state.get('status')}")

try:
    display(Image(app.get_graph().draw_png()))
    print("Graph visualized. Note that 'final_step' does not point to END.")
except Exception as e:
    print(f"Graph visualization failed: {e}")
``````
</details>

<details><summary>解説009</summary>

#### この問題のポイント
*   **学習内容:** グラフの終点として必ずしも `END` を明示的に指定する必要はなく、あるノードから先に遷移するエッジがなければ、そのノードの処理が終わった時点でグラフの実行が停止し、その時点での状態が最終状態となることを理解します。ただし、これはLangGraphの挙動の一つであり、デバッグや可視化、他のツールとの連携（例: Langfuse）を考慮すると、可能な限りグラフの終点を `END` に接続することが推奨されます。
*   **コード解説:**
    *   `final_processing_node`を作成し、このノードから `END` へのエッジ（`workflow.add_edge("final_step", END)`）を定義していません。
    *   `app.stream()` を使ってグラフを実行すると、`final_processing_node` が実行された後、それ以上進むべきノードがないため、処理が停止します。`stream()` の最後の出力（この場合は `final_processing_node` の出力）が、その実行における最終的な状態を示します。
    *   `app.invoke()` の場合、グラフが明示的に `END` に到達しないと、バージョンや設定によってはエラーが発生したり、予期しない挙動をしたりする可能性があります。一般的に `invoke()` はグラフが `END` に到達し、完全な最終状態が確定することを期待します。この問題では、主に `stream()` での挙動を確認し、`invoke()` は参考として示しています。
    *   可視化すると、`final_step` ノードから `END` (または他のノード) への矢印がないことが確認できます。
*   **重要な注意点:**
    *   **`END` の使用推奨:** LangGraphでは、グラフの論理的な終了点を明確にするために `END` を使用することが強く推奨されます。これにより、グラフの構造が理解しやすくなり、デバッグも容易になります。また、LangSmith/Langfuseのようなトレースツールは、`END` への到達をもって一連の処理の完了とみなすことが多いため、連携時にも重要です。
    *   **`compile(check_interruptions=True)`:** グラフをコンパイルする際に `check_interruptions=True` を指定すると、中断（Interrupt）が発生しない限り、グラフが必ず `END` に到達することを強制できます。`END` に到達しないパスがある場合、コンパイル時または実行時にエラーが発生します。
    *   この問題は、`END` を使わない場合の挙動を理解するためのものであり、実際の開発では `END` を適切に配置する設計を心がけてください。
---
</details>

### ■ 問題010: LLMノードと非LLMノードの連携強化

LLM（大規模言語モデル）を組み込んだノードと、LLM以外の処理を行うノード（例: 文字列操作、データ抽出など）を連携させる方法を学びましょう。ここでは、LLMに質問を投げて得られた応答（文字列）から、特定の情報を抽出・加工して状態を更新する、より実践的なグラフを作成します。

In [ ]:
# 解答欄010
from typing import TypedDict, Annotated
import re # 正規表現モジュールをインポート
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class ExtractionState(TypedDict):
    messages: Annotated[list, add_messages]
    user_question: str # ユーザーの元の質問
    llm_response_text: str # LLMの生の応答テキスト
    extracted_info: str | None # LLMの応答から抽出された情報

# --- ノード定義 (Nodes) ---
def get_user_question(state: ExtractionState):
    # ユーザーの質問を状態に保存
    last_message_content = state["messages"][-1].content
    print(f"get_user_question: Received question: '{last_message_content}'")
    return {"user_question": ____} # last_message_content

def llm_responder_node(state: ExtractionState):
    # LLMに質問を投げるノード
    question = state["user_question"]
    print(f"llm_responder_node: Asking LLM: '{question}'")
    # LLMに渡すメッセージは、過去の履歴全体でも、最新の質問だけでも良い
    # ここでは簡単のため、最新の質問のみをHumanMessageとして渡す
    response = llm.invoke([HumanMessage(content=question)])
    response_content = response.content
    print(f"llm_responder_node: LLM raw response: '{response_content}'")
    return {"messages": [response], "llm_response_text": ____} # response_content

def data_extractor_node(state: ExtractionState):
    # LLMの応答から情報を抽出するノード
    raw_response = state["llm_response_text"]
    # 例: LLMが「日本の首都は東京です。」と答えたら「東京」を抽出
    # ここでは簡単な正規表現で「XXはYYです」のYY部分を抽出試行
    extracted = None
    match = re.search(r"(?:は|is)\s*([^。.]+)[.。]?", raw_response) # 簡易的な抽出
    if match:
        extracted = match.group(1).strip()
    
    print(f"data_extractor_node: Raw response: '{raw_response}', Extracted: '{extracted}'")
    return {"extracted_info": extracted, "messages": [AIMessage(content=f"Extracted: {extracted}")]}

# --- グラフ構築 (Graph) ---
workflow = ____(ExtractionState) # StateGraph

workflow.____("capture_question", get_user_question) # add_node
workflow.____("ask_llm", llm_responder_node) # add_node
workflow.____("extract_data", data_extractor_node) # add_node

workflow.____("capture_question") # set_entry_point

workflow.____("capture_question", "ask_llm") # add_edge
workflow.____("ask_llm", "extract_data") # add_edge
workflow.____("extract_data", ____) # add_edge, END

app = workflow.____() # compile

# --- グラフの実行と結果表示 ---
questions = [
    "日本の首都は何ですか？",
    "フランスの有名な画家の名前を一人教えてください。",
    "今日の天気は？" # これは抽出パターンにマッチしない可能性
]

for q_text in questions:
    print(f"\n--- LLM連携と情報抽出テスト (質問: {q_text}) ---")
    inputs = {"messages": [HumanMessage(content=q_text)]}
    final_state = None
    for s in app.____(inputs, {"recursion_limit": 5}): # stream
        print(s)
        if "extract_data" in s:
            final_state = s["extract_data"]

    if final_state:
        print(f"Final State: {final_state}")
        print(f"  User Question: {final_state.get('user_question')}")
        print(f"  LLM Response: {final_state.get('llm_response_text')}")
        print(f"  Extracted Info: {final_state.get('extracted_info')}")
    else:
        # streamの最後がENDだった場合など、特定のノード名で状態が取れない場合がある
        # その場合はinvokeで最終状態を取得
        final_state_invoked = app.invoke(inputs, {"recursion_limit": 5})
        print(f"Final State (from invoke): {final_state_invoked}")
        print(f"  User Question: {final_state_invoked.get('user_question')}")
        print(f"  LLM Response: {final_state_invoked.get('llm_response_text')}")
        print(f"  Extracted Info: {final_state_invoked.get('extracted_info')}")

try:
    display(Image(app.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")

<details><summary>解答010</summary>

``````python
# 解答010
from typing import TypedDict, Annotated
import re # 正規表現モジュールをインポート
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class ExtractionState(TypedDict):
    messages: Annotated[list, add_messages]
    user_question: str # ユーザーの元の質問
    llm_response_text: str # LLMの生の応答テキスト
    extracted_info: str | None # LLMの応答から抽出された情報

# --- ノード定義 (Nodes) ---
def get_user_question(state: ExtractionState):
    # ユーザーの質問を状態に保存
    last_message_content = state["messages"][-1].content
    print(f"get_user_question: Received question: '{last_message_content}'")
    return {"user_question": last_message_content}

def llm_responder_node(state: ExtractionState):
    # LLMに質問を投げるノード
    question = state["user_question"]
    print(f"llm_responder_node: Asking LLM: '{question}'")
    response = llm.invoke([HumanMessage(content=question)])
    response_content = response.content
    print(f"llm_responder_node: LLM raw response: '{response_content}'")
    return {"messages": [response], "llm_response_text": response_content}

def data_extractor_node(state: ExtractionState):
    # LLMの応答から情報を抽出するノード
    raw_response = state["llm_response_text"]
    extracted = None
    # 改善された正規表現: 「XXはYYです」「XX is YY」のようなパターンや、単に「YYです」のような応答にも対応試行
    # 質問が「日本の首都は何ですか？」で応答が「東京です。」の場合「東京」を抽出
    # 質問が「日本の首都は？」で応答が「東京」の場合「東京」を抽出
    patterns = [
        r"(?:.+は|.+\s*is)\s*(.+?)(?:です|。|\.|\s*for|$)", #「～はXです」「～ is X」
        r"^([^。.]+?)(?:です|。|\.|\s*for|$)" # 文頭から「Xです」
    ]
    for pattern in patterns:
        match = re.search(pattern, raw_response)
        if match:
            extracted = match.group(1).strip()
            if extracted.lower() == state["user_question"].lower().replace("何ですか","").replace("何","зиру").strip("?？") : # 質問自体が答えになるような場合を除外
                 extracted = None # 例：「天気は？」->「晴れです」はOKだが、「天気は？」->「天気」はNG
                 continue
            break
    
    # もし上記で抽出できなかった場合、LLMの応答が単語やフレーズそのものである可能性を考慮
    if not extracted and len(raw_response.split()) < 5 and not state["user_question"].startswith(raw_response): # 短い応答で、質問の繰り返しでない場合
        extracted = raw_response.strip()

    print(f"data_extractor_node: Raw response: '{raw_response}', Extracted: '{extracted}'")
    return {"extracted_info": extracted, "messages": [AIMessage(content=f"Extracted info: {extracted if extracted else 'N/A'}")]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(ExtractionState)

workflow.add_node("capture_question", get_user_question)
workflow.add_node("ask_llm", llm_responder_node)
workflow.add_node("extract_data", data_extractor_node)

workflow.set_entry_point("capture_question")

workflow.add_edge("capture_question", "ask_llm")
workflow.add_edge("ask_llm", "extract_data")
workflow.add_edge("extract_data", END)

app = workflow.compile()

# --- グラフの実行と結果表示 ---
questions = [
    "日本の首都は何ですか？",
    "フランスの有名な画家の名前を一人教えてください。", # LLMの回答次第で抽出成功/失敗が変わる
    "1+1は何？", # LLMが「2です」と答えれば抽出できるかも
    "今日の天気は？" # 「晴れです」なら「晴れ」を抽出期待
]

for q_text in questions:
    print(f"\n--- LLM連携と情報抽出テスト (質問: {q_text}) ---")
    inputs = {"messages": [HumanMessage(content=q_text)], "llm_response_text": "", "extracted_info": None} # 初期値を設定
    
    # streamで実行し、各ステップの状態変化を確認
    # final_state_from_stream = {} # streamの最終的な状態を保持する辞書
    # for s_chunk in app.stream(inputs, {"recursion_limit": 5}):
    #     print(s_chunk)
    #     final_state_from_stream.update(s_chunk)
    # final_state_data = final_state_from_stream.get(list(final_state_from_stream.keys())[-1]) # 最後のキーのデータ

    # invokeで最終状態を取得する方がシンプル
    final_state_data = app.invoke(inputs, {"recursion_limit": 5})

    if final_state_data:
        print(f"Final State: {final_state_data}")
        print(f"  User Question: {final_state_data.get('user_question')}")
        print(f"  LLM Response: {final_state_data.get('llm_response_text')}")
        print(f"  Extracted Info: {final_state_data.get('extracted_info')}")
    else:
        print("Could not retrieve final state.")

try:
    display(Image(app.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")
``````
</details>

<details><summary>解説010</summary>

#### この問題のポイント
*   **学習内容:** LLMを呼び出すノードと、その出力を処理する非LLMノードを組み合わせることで、より高度な情報処理パイプラインを構築する方法を学びます。具体的には、LLMの自然言語応答から正規表現などを用いて構造化された情報を抽出し、グラフの状態を更新します。
*   **コード解説:**
    *   `ExtractionState`には、ユーザーの質問 (`user_question`)、LLMの生の応答テキスト (`llm_response_text`)、そして抽出された情報 (`extracted_info`) を保持するためのキーが定義されています。
    *   `get_user_question`ノード: ユーザーからの最初のメッセージを `user_question` として状態に保存します。
    *   `llm_responder_node`: 保存された `user_question` を使ってLLMに問い合わせを行い、得られた応答を `messages` (AIMessageとして) と `llm_response_text` (生の文字列として) 状態に保存します。
    *   `data_extractor_node`: `llm_response_text` から情報を抽出します。この例では、簡単な正規表現 `re.search(r"(?:は|is)\s*([^。.]+)[.。]?", raw_response)` を使用して、「AはBです」や「A is B」といった形式の文からBの部分を抽出しようと試みています。抽出結果は `extracted_info` として状態に保存されます。正規表現は完璧ではなく、LLMの応答形式によってはうまく抽出できない場合もありますが、ここではLLMの出力後処理の一例として示しています。
    *   グラフは `capture_question` -> `ask_llm` -> `extract_data` -> `END` というシーケンシャルな流れで定義されています。
    *   実行時には、異なる質問を投げ、LLMの応答とそこから抽出された情報（または抽出できなかった場合は `None` や `N/A`）が最終状態に含まれることを確認します。
*   **発展:**
    *   情報抽出の方法は正規表現に限らず、より高度なNLPライブラリ（例: spaCy）や、LangChainが提供するOutput Parsers、あるいは別のLLMコール（Function Calling/Tool Calling対応モデルならより高精度）を使って行うことも考えられます。
    *   抽出に失敗した場合のフォールバック処理（例: ユーザーに再確認を求める、デフォルト値を設定するなど）をグラフに追加することもできます。
---
</details>

### ■ 問題011: 状態を使ったループ（シンプルなカウンター制御）

グラフ内で特定の条件が満たされるまで処理を繰り返す「ループ」構造を、状態と条件付きエッジを使って実現する方法を学びましょう。ここでは、カウンターが指定した上限値に達するまで、カウンターを増やし続けるシンプルなループ処理を持つグラフを作成します。これは、より複雑な自己修正ループや反復処理の基礎となります。

In [ ]:
# 解答欄011
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
class LoopState(TypedDict):
    messages: Annotated[list, add_messages]
    current_count: int # 現在のカウンター値
    max_count: int     # ループを終了するカウンターの上限値

# --- ノード定義 (Nodes) ---
def increment_and_log(state: LoopState):
    # カウンターを1増やし、メッセージをログに記録する
    count = state.get("current_count", 0) + 1
    log_message = f"Counter incremented to {count}."
    print(f"increment_and_log: {log_message}")
    return {"current_count": count, "messages": [AIMessage(content=log_message)]}

def final_log(state: LoopState):
    # ループ終了後に最終メッセージをログに記録する
    log_message = f"Loop finished. Final count is {state['current_count']}. Max count was {state['max_count']}."
    print(f"final_log: {log_message}")
    return {"messages": [AIMessage(content=log_message)]}

# --- 条件付きエッジのルーター関数 ---
def should_continue_loop(state: LoopState):
    # current_countがmax_count未満ならループを継続、そうでなければ終了
    current = state.get("current_count", 0)
    max_c = state.get("max_count", 0)
    print(f"should_continue_loop: Current: {current}, Max: {max_c}")
    if current < max_c:
        print("  -> Routing to 'increment_and_log' (continue loop)")
        return ____ # "continue_loop"
    else:
        print("  -> Routing to 'exit_loop' (end loop)")
        return ____ # "exit_loop"

# --- グラフ構築 (Graph) ---
workflow = ____(LoopState) # StateGraph

workflow.____("increment_logger", increment_and_log) # add_node
workflow.____("final_logger", final_log) # add_node

# エントリポイントは直接 increment_logger とする (初期カウントは入力で設定)
workflow.____("increment_logger") # set_entry_point

# 条件付きエッジでループを制御
workflow.____( # add_conditional_edges
    "increment_logger", # このノードの後に条件分岐
    ____, # should_continue_loop (ルーター関数)
    {
        "continue_loop": "increment_logger", # "continue_loop"なら再度increment_loggerへ (ループバック)
        "exit_loop": "final_logger"      # "exit_loop"ならfinal_loggerへ
    }
)

# final_loggerノードからENDへ
workflow.____("final_logger", ____) # add_edge, END

app = workflow.____() # compile

# --- グラフの実行と結果表示 ---
max_iterations = 3
print(f"\n--- ループテスト (max_count = {max_iterations}) ---")
inputs = {
    "messages": [HumanMessage(content=f"Start loop up to {max_iterations}")],
    "current_count": 0, # 初期カウント
    "max_count": max_iterations
}

for s in app.____(inputs, {"recursion_limit": 10}): # stream (recursion_limitに注意)
    print(s)

final_state = app.____(inputs, {"recursion_limit": 10}) # invoke
print(f"Final State: {final_state}")
assert final_state["current_count"] == max_iterations
assert f"Loop finished. Final count is {max_iterations}." in final_state["messages"][-1].content
print("Loop test passed.")

try:
    display(Image(app.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")

<details><summary>解答011</summary>

``````python
# 解答011
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class LoopState(TypedDict):
    messages: Annotated[list, add_messages]
    current_count: int # 現在のカウンター値
    max_count: int     # ループを終了するカウンターの上限値

# --- ノード定義 (Nodes) ---
def increment_and_log(state: LoopState):
    # カウンターを1増やし、メッセージをログに記録する
    count = state.get("current_count", 0) + 1
    log_message = f"Counter incremented to {count}."
    print(f"increment_and_log: {log_message}")
    return {"current_count": count, "messages": [AIMessage(content=log_message)]}

def final_log(state: LoopState):
    # ループ終了後に最終メッセージをログに記録する
    log_message = f"Loop finished. Final count is {state['current_count']}. Max count was {state['max_count']}."
    print(f"final_log: {log_message}")
    return {"messages": [AIMessage(content=log_message)]}

# --- 条件付きエッジのルーター関数 ---
def should_continue_loop(state: LoopState):
    # current_countがmax_count未満ならループを継続、そうでなければ終了
    current = state.get("current_count", 0)
    max_c = state.get("max_count", 0)
    print(f"should_continue_loop: Current: {current}, Max: {max_c}")
    if current < max_c:
        print("  -> Routing to 'increment_and_log' (continue loop)")
        return "continue_loop"
    else:
        print("  -> Routing to 'exit_loop' (end loop)")
        return "exit_loop"

# --- グラフ構築 (Graph) ---
workflow = StateGraph(LoopState)

workflow.add_node("increment_logger", increment_and_log)
workflow.add_node("final_logger", final_log)

# エントリポイントは直接 increment_logger とする (初期カウントは入力で設定)
workflow.set_entry_point("increment_logger")

# 条件付きエッジでループを制御
workflow.add_conditional_edges(
    "increment_logger", # このノードの後に条件分岐
    should_continue_loop, # ルーター関数
    {
        "continue_loop": "increment_logger", # "continue_loop"なら再度increment_loggerへ (ループバック)
        "exit_loop": "final_logger"      # "exit_loop"ならfinal_loggerへ
    }
)

# final_loggerノードからENDへ
workflow.add_edge("final_logger", END)

app = workflow.compile()

# --- グラフの実行と結果表示 ---
max_iterations = 3
print(f"\n--- ループテスト (max_count = {max_iterations}) ---")
inputs = {
    "messages": [HumanMessage(content=f"Start loop up to {max_iterations}")],
    "current_count": 0, # 初期カウント
    "max_count": max_iterations
}

for s in app.stream(inputs, {"recursion_limit": 10}): # stream (recursion_limitに注意)
    print(s)

final_state = app.invoke(inputs, {"recursion_limit": 10}) # invoke
print(f"Final State: {final_state}")
assert final_state["current_count"] == max_iterations
assert f"Loop finished. Final count is {max_iterations}." in final_state["messages"][-1].content
print("Loop test passed.")

try:
    display(Image(app.get_graph().draw_png()))
    print("Graph visualized. Note the loop from 'increment_logger' back to itself.")
except Exception as e:
    print(f"Graph visualization failed: {e}")
``````
</details>

<details><summary>解説011</summary>

#### この問題のポイント
*   **学習内容:** 状態(`State`)と条件付きエッジ(`add_conditional_edges`)を組み合わせて、グラフ内にループ構造（繰り返し処理）を実装する方法を学びます。具体的には、カウンターが上限に達するまで特定のノード（ここでは`increment_logger`）を繰り返し実行し、条件を満たしたら別のノード（`final_logger`）に遷移して終了します。
*   **コード解説:**
    *   `LoopState`には、現在のカウンター値 `current_count` とループの終了条件となる `max_count` が定義されています。
    *   `increment_and_log`ノード: `current_count`を1増やし、その結果をログ（`messages`）に記録します。
    *   `final_log`ノード: ループが終了した際に最終的なメッセージをログに記録します。
    *   `should_continue_loop`ルーター関数: `current_count`が`max_count`未満であれば `"continue_loop"` を返し、そうでなければ `"exit_loop"` を返します。
    *   グラフ構築部分:
        *   エントリポイントは `increment_logger` です。
        *   `add_conditional_edges` を使って `increment_logger` の後に `should_continue_loop` ルーターを実行します。
        *   ルーターが `"continue_loop"` を返した場合、処理は再び `increment_logger` に戻ります（これがループバックエッジとなり、ループを形成します）。
        *   ルーターが `"exit_loop"` を返した場合、処理は `final_logger` に進み、その後 `END` に到達してグラフが終了します。
    *   実行時には、`inputs`で `current_count` の初期値（例: 0）と `max_count` を設定します。グラフは `increment_logger` を `max_count` 回実行し、その後 `final_logger` を実行して終了します。
    *   `app.stream()` や `app.invoke()` を呼び出す際には、`recursion_limit`（再帰深度の上限）を適切に設定する必要があります。ループが深い場合、デフォルトの再帰上限を超える可能性があるためです。
*   **応用:** このループ構造は、例えばLLMに繰り返し改善を指示したり、特定の条件が満たされるまでツールを実行し続けたりするなど、より複雑なエージェント的挙動の基礎となります。
---
</details>

### ■ 問題012: グラフの入力と出力のカスタマイズと明確化

LangGraphのグラフを実行する際、`invoke()` や `stream()` メソッドに渡す初期状態の構造と、グラフ全体の最終的な出力状態の構造を意識することが重要です。`StateGraph` に渡す状態クラス（例: `TypedDict`）の定義が、実質的にグラフの入力と出力のスキーマ（型定義）となります。この問題では、入力として複数の情報を受け取り、それらを処理して特定の構造で出力するグラフを作成し、入出力の対応関係を明確に意識します。

In [ ]:
# 解答欄012
from typing import TypedDict, Annotated, List, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (入力と出力のスキーマを兼ねる) ---
class ProcessedData(TypedDict):
    item_id: str
    description: str
    is_processed: bool

class ComplexIOState(TypedDict):
    # 入力として期待されるキー
    raw_item_name: str
    raw_item_details: List[str]
    processing_threshold: Optional[int]

    # 処理中に使われるキー
    messages: Annotated[list, add_messages] # 処理ログ用
    internal_counter: int

    # 出力として期待される主要なキー
    processed_data: Optional[ProcessedData] # 処理結果
    error_message: Optional[str] # エラー発生時のメッセージ

# --- ノード定義 (Nodes) ---
def initialize_processing(state: ComplexIOState):
    print(f"initialize_processing: Received item '{state['raw_item_name']}' with details {state['raw_item_details']}")
    threshold = state.get("processing_threshold", 0)
    return {
        "messages": [AIMessage(content=f"Initializing processing for {state['raw_item_name']}")],
        "internal_counter": 0,
        "processed_data": None, # 出力キーを初期化
        "error_message": None   # 出力キーを初期化
        # processing_threshold は入力からそのまま引き継がれる
    }

def main_processor(state: ComplexIOState):
    name = state["raw_item_name"]
    details_count = len(state["raw_item_details"])
    counter = state["internal_counter"] + 1
    threshold = state.get("processing_threshold", 0)
    
    log_msg = f"Processing '{name}', detail count: {details_count}, attempt: {counter}, threshold: {threshold}"
    print(f"main_processor: {log_msg}")

    if details_count == 0:
        err_msg = "No details provided."
        return {
            "messages": [AIMessage(content=f"Error: {err_msg}")],
            "error_message": err_msg,
            "internal_counter": counter
        }
    
    # 簡単な処理のシミュレーション
    if counter > threshold: # processing_threshold を超えたら処理成功とするデモ
        processed_item = ProcessedData(
            item_id=f"PROC_{name.upper()}_{counter}",
            description=f"Successfully processed {name} with {details_count} details after {counter} attempts.",
            is_processed=True
        )
        return {
            "messages": [AIMessage(content=f"Successfully processed {name}")],
            "processed_data": ____, # processed_item
            "internal_counter": counter
        }
    else:
        return {
            "messages": [AIMessage(content=f"Attempt {counter} for {name} did not meet threshold.")],
            "internal_counter": counter
        }

# --- 条件付きエッジのルーター関数 ---
def check_status(state: ComplexIOState):
    if state.get("error_message"):
        print("check_status: Error detected, routing to END.")
        return "__end__" # エラーなら即終了 (ENDに直接マッピングされる特別な名前)
    if state.get("processed_data") and state["processed_data"]["is_processed"]:
        print("check_status: Successfully processed, routing to END.")
        return ____ # "__end__"
    else:
        print("check_status: Not yet processed or error, routing back to main_processor.")
        return ____ # "retry_processing"

# --- グラフ構築 (Graph) ---
workflow = ____(ComplexIOState) # StateGraph

workflow.____("initializer", initialize_processing) # add_node
workflow.____("processor", main_processor) # add_node

workflow.____("initializer") # set_entry_point
workflow.____("initializer", "processor") # add_edge

workflow.____( # add_conditional_edges
    "processor",
    check_status,
    {
        "__end__": END, # ルーターが"__end__"を返したらグラフ終了
        "retry_processing": "processor" # "retry_processing"なら再度processorへ (ループ)
    }
)

app = workflow.____() # compile

# --- グラフの実行と結果表示 ---
inputs_success = {
    "raw_item_name": "TestItem1",
    "raw_item_details": ["detail A", "detail B"],
    "processing_threshold": 2 # 3回目のprocessor呼び出しで成功するはず
}

inputs_fail_no_details = {
    "raw_item_name": "TestItem2",
    "raw_item_details": [], # 詳細なし -> エラー
    "processing_threshold": 1
}

test_cases = {"Success Case": inputs_success, "Failure Case (No Details)": inputs_fail_no_details}

for case_name, inputs_data in test_cases.items():
    print(f"\n--- I/Oカスタマイズテスト: {case_name} ---")
    # 入力時には、Stateで定義された必須キーとオプショナルなキーを渡す
    # messages, internal_counter, processed_data, error_message は処理中に設定されるので入力不要
    current_inputs = inputs_data.copy()
    # messagesはadd_messagesの仕様上、リストでないとエラーになるため、空リストで初期化することが安全
    current_inputs.setdefault("messages", []) 

    final_output_state = app.invoke(current_inputs, {"recursion_limit": 10})
    print(f"Final Output State for {case_name}: {final_output_state}")

    # 出力状態の確認 (processed_data または error_message が期待通りか)
    if final_output_state.get("processed_data"):
        print(f"  Processed Item ID: {final_output_state['processed_data']['item_id']}")
        print(f"  Processed: {final_output_state['processed_data']['is_processed']}")
    if final_output_state.get("error_message"):
        print(f"  Error: {final_output_state['error_message']}")

try:
    display(Image(app.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")

<details><summary>解答012</summary>

``````python
# 解答012
from typing import TypedDict, Annotated, List, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (入力と出力のスキーマを兼ねる) ---
class ProcessedData(TypedDict):
    item_id: str
    description: str
    is_processed: bool

class ComplexIOState(TypedDict):
    # 入力として期待されるキー
    raw_item_name: str
    raw_item_details: List[str]
    processing_threshold: Optional[int]

    # 処理中に使われるキー
    messages: Annotated[list, add_messages] # 処理ログ用
    internal_counter: int

    # 出力として期待される主要なキー
    processed_data: Optional[ProcessedData] # 処理結果
    error_message: Optional[str] # エラー発生時のメッセージ

# --- ノード定義 (Nodes) ---
def initialize_processing(state: ComplexIOState):
    print(f"initialize_processing: Received item '{state['raw_item_name']}' with details {state['raw_item_details']}")
    # processing_thresholdが入力で与えられていない場合のデフォルト値を設定
    # この例では、state.getで取得する際にデフォルト値を設定するアプローチではなく、
    # 入力時に processing_threshold が Optional であることを明示し、
    # main_processor で利用する際に .get でデフォルト値を扱う。
    # ここでは、入力された値をそのまま引き継ぎ、他の内部状態を初期化。
    return {
        "messages": [AIMessage(content=f"Initializing processing for {state['raw_item_name']}")],
        "internal_counter": 0,
        "processed_data": None, # 出力キーを初期化
        "error_message": None   # 出力キーを初期化
    }

def main_processor(state: ComplexIOState):
    name = state["raw_item_name"]
    details_count = len(state["raw_item_details"])
    counter = state["internal_counter"] + 1
    # processing_threshold が None の場合はデフォルト値 0 を使用
    threshold = state.get("processing_threshold") if state.get("processing_threshold") is not None else 0
    
    log_msg = f"Processing '{name}', detail count: {details_count}, attempt: {counter}, threshold: {threshold}"
    print(f"main_processor: {log_msg}")

    if details_count == 0:
        err_msg = "No details provided."
        return {
            "messages": [AIMessage(content=f"Error: {err_msg}")],
            "error_message": err_msg,
            "internal_counter": counter
        }
    
    if counter > threshold:
        processed_item = ProcessedData(
            item_id=f"PROC_{name.upper()}_{counter}",
            description=f"Successfully processed {name} with {details_count} details after {counter} attempts.",
            is_processed=True
        )
        return {
            "messages": [AIMessage(content=f"Successfully processed {name}")],
            "processed_data": processed_item,
            "internal_counter": counter
        }
    else:
        return {
            "messages": [AIMessage(content=f"Attempt {counter} for {name} did not meet threshold.")],
            "internal_counter": counter
        }

# --- 条件付きエッジのルーター関数 ---
def check_status(state: ComplexIOState):
    if state.get("error_message"):
        print("check_status: Error detected, routing to END.")
        return "__end__" 
    if state.get("processed_data") and state["processed_data"]["is_processed"]:
        print("check_status: Successfully processed, routing to END.")
        return "__end__"
    else:
        print("check_status: Not yet processed or error, routing back to main_processor.")
        return "retry_processing"

# --- グラフ構築 (Graph) ---
workflow = StateGraph(ComplexIOState)

workflow.add_node("initializer", initialize_processing)
workflow.add_node("processor", main_processor)

workflow.set_entry_point("initializer")
workflow.add_edge("initializer", "processor")

workflow.add_conditional_edges(
    "processor",
    check_status,
    {
        "__end__": END,
        "retry_processing": "processor"
    }
)

app = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs_success = {
    "raw_item_name": "TestItem1",
    "raw_item_details": ["detail A", "detail B"],
    "processing_threshold": 2 
}

inputs_fail_no_details = {
    "raw_item_name": "TestItem2",
    "raw_item_details": [], 
    "processing_threshold": 1
}

inputs_optional_threshold_not_provided = {
    "raw_item_name": "TestItem3",
    "raw_item_details": ["detail C"],
    # processing_threshold は提供しない (OptionalなのでOK)
}

test_cases = {
    "Success Case": inputs_success,
    "Failure Case (No Details)": inputs_fail_no_details,
    "Success Case (Threshold Not Provided)": inputs_optional_threshold_not_provided
}

for case_name, inputs_data in test_cases.items():
    print(f"\n--- I/Oカスタマイズテスト: {case_name} ---")
    current_inputs = inputs_data.copy()
    current_inputs.setdefault("messages", []) 
    # Optionalでない内部状態キーで、初期化ノードで設定されるものは入力時に含めなくても良い
    # 例: internal_counter, processed_data, error_message

    final_output_state = app.invoke(current_inputs, {"recursion_limit": 10})
    print(f"Final Output State for {case_name}: {final_output_state}")

    if final_output_state.get("processed_data"):
        print(f"  Processed Item ID: {final_output_state['processed_data']['item_id']}")
        print(f"  Processed: {final_output_state['processed_data']['is_processed']}")
    if final_output_state.get("error_message"):
        print(f"  Error: {final_output_state['error_message']}")

try:
    display(Image(app.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")
``````
</details>

<details><summary>解説012</summary>

#### この問題のポイント
*   **学習内容:** `TypedDict` を使ってグラフの状態スキーマを定義する際、どのキーがグラフへの主要な「入力」として期待され、どのキーがグラフからの主要な「出力」として扱われるのかを明確に意識することの重要性を学びます。また、`Optional`型やデフォルト値の扱い方についても触れます。
*   **コード解説:**
    *   `ComplexIOState` (状態スキーマ):
        *   **入力想定キー:** `raw_item_name`, `raw_item_details`, `processing_threshold` (これは `Optional` なので、入力時に省略可能)。これらはグラフ実行時に `invoke` や `stream` の `inputs` 引数で渡されることが期待されます。
        *   **処理中キー:** `messages`, `internal_counter`。これらは主にグラフ内部の処理やログのために使われ、通常は入力時に指定しません（`messages` は `add_messages` のために空リストで初期化することがあります）。
        *   **出力想定キー:** `processed_data` (処理成功時の結果), `error_message` (エラー発生時の情報)。これらのキーの値が、グラフ実行後の最終的な成果物となります。
    *   `initialize_processing`ノード: 入力値を受け取り、処理に必要な内部状態（`internal_counter`など）や出力用キー（`processed_data`, `error_message`）を初期化します。
    *   `main_processor`ノード: 主要な処理ロジックを担当します。入力された `raw_item_name` や `raw_item_details`、そして `processing_threshold` (入力されなければデフォルト値を使用) に基づいて処理を行い、成功すれば `processed_data` を、失敗すれば `error_message` を更新します。また、処理試行回数を `internal_counter` で管理します。
    *   `check_status`ルーター関数: `error_message` があれば終了。`processed_data` があれば終了。それ以外（まだ処理が完了していない、またはエラーではないが成功もしていない）場合は `main_processor` に戻って処理を続行（リトライ/ループ）します。
    *   グラフ実行時:
        *   `inputs`辞書には、`ComplexIOState`で定義した入力想定キー（`raw_item_name`など）を指定します。
        *   `invoke()` から返される `final_output_state` は、`ComplexIOState` と同じ構造の辞書です。この中から出力想定キー（`processed_data` や `error_message`）の値を確認することで、グラフの実行結果を得ます。
*   **重要な点:**
    *   状態スキーマ（`TypedDict`）は、グラフのインターフェース（入力と出力の形式）を定義する上で中心的な役割を果たします。
    *   入力時にどのキーが必要で、どのキーがオプショナルか、そしてグラフ実行後にどのキーに出力結果が格納されるのかを明確に設計することが、再利用可能で理解しやすいグラフを作る上で重要です。
---
</details>

### ■ 問題013: 複数のLLM呼び出しを含むグラフ

一つのグラフ内で、異なる役割やプロンプトを持つ複数のLLM呼び出しノードを組み込む方法を学びましょう。例えば、最初のLLMがアイデアを生成し、次のLLMがそのアイデアを評価・洗練する、といった連携が考えられます。この問題では、簡単な役割分担を持つ2つのLLMノードを直列に接続します。

In [ ]:
# 解答欄013
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class MultiLLMState(TypedDict):
    messages: Annotated[list, add_messages]
    original_topic: str # ユーザーからの最初のトピック
    generated_idea: str | None # アイデア生成LLMの出力
    evaluated_idea: str | None # アイデア評価LLMの出力

# --- ノード定義 (Nodes) ---
def get_topic(state: MultiLLMState):
    topic = state["messages"][-1].content
    print(f"get_topic: Original topic is '{topic}'")
    return {"original_topic": topic, "messages": [AIMessage(content=f"Topic received: {topic}")]}

def idea_generation_node(state: MultiLLMState):
    topic = state["original_topic"]
    print(f"idea_generation_node: Generating idea for topic: '{topic}'")
    
    # このLLM用のプロンプトテンプレート
    prompt_template_idea = ChatPromptTemplate.from_messages([
        ("system", "あなたは新しいアイデアを生み出すのが得意なAIです。与えられたトピックに関して、ユニークで面白いアイデアを一つ提案してください。アイデアは簡潔に一行で述べてください。"),
        ("human", "トピック: {topic}")
    ])
    
    # llm変数を使ってプロンプトを実行 (ノートブック冒頭で初期化された共通のllm)
    chain = prompt_template_idea | llm 
    response = chain.invoke({"topic": topic})
    idea = response.content.strip()
    
    print(f"idea_generation_node: Generated idea: '{idea}'")
    return {"generated_idea": idea, "messages": [AIMessage(content=f"Generated Idea: {idea}")]}

def idea_evaluation_node(state: MultiLLMState):
    idea = state["generated_idea"]
    if not idea:
        return {"messages": [AIMessage(content="No idea to evaluate.")], "evaluated_idea": "N/A"}
        
    print(f"idea_evaluation_node: Evaluating idea: '{idea}'")
    
    # このLLM用のプロンプトテンプレート
    prompt_template_eval = ChatPromptTemplate.from_messages([
        ("system", "あなたはアイデアを客観的に評価するのが得意なAIです。与えられたアイデアについて、その実現可能性と面白さを評価し、短いコメントを述べてください。"),
        ("human", "評価対象のアイデア: {idea}")
    ])
    
    chain = ____ | llm # prompt_template_eval
    response = chain.invoke({"idea": idea})
    evaluation = response.content.strip()
    
    print(f"idea_evaluation_node: Evaluation: '{evaluation}'")
    return {"evaluated_idea": evaluation, "messages": [AIMessage(content=f"Evaluation: {evaluation}")]}

# --- グラフ構築 (Graph) ---
workflow = ____(MultiLLMState) # StateGraph

workflow.____("capture_topic", get_topic) # add_node
workflow.____("generate_idea", idea_generation_node) # add_node
workflow.____("evaluate_idea", idea_evaluation_node) # add_node

workflow.____("capture_topic") # set_entry_point

workflow.____("capture_topic", "generate_idea") # add_edge
workflow.____("generate_idea", "evaluate_idea") # add_edge
workflow.____("evaluate_idea", ____) # add_edge, END

app = workflow.____() # compile

# --- グラフの実行と結果表示 ---
topics_to_test = [
    "新しい料理のレシピ",
    "未来の交通手段",
    "週末の過ごし方"
]

for topic_text in topics_to_test:
    print(f"\n--- 複数LLM連携テスト (トピック: {topic_text}) ---")
    inputs = {
        "messages": [HumanMessage(content=topic_text)],
        "original_topic": "", # 初期化ノードで設定される
        "generated_idea": None,
        "evaluated_idea": None
    }
    final_state = app.invoke(inputs, {"recursion_limit": 5})
    print(f"Final State for topic '{topic_text}':")
    print(f"  Original Topic: {final_state.get('original_topic')}")
    print(f"  Generated Idea: {final_state.get('generated_idea')}")
    print(f"  Evaluated Idea: {final_state.get('evaluated_idea')}")
    # print(f"  Message History: {final_state.get('messages')}") # 必要なら表示

try:
    display(Image(app.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")

<details><summary>解答013</summary>

``````python
# 解答013
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import Image, display

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class MultiLLMState(TypedDict):
    messages: Annotated[list, add_messages]
    original_topic: str # ユーザーからの最初のトピック
    generated_idea: str | None # アイデア生成LLMの出力
    evaluated_idea: str | None # アイデア評価LLMの出力

# --- ノード定義 (Nodes) ---
def get_topic(state: MultiLLMState):
    topic = state["messages"][-1].content
    print(f"get_topic: Original topic is '{topic}'")
    return {"original_topic": topic, "messages": [AIMessage(content=f"Topic received: {topic}")]}

def idea_generation_node(state: MultiLLMState):
    topic = state["original_topic"]
    print(f"idea_generation_node: Generating idea for topic: '{topic}'")
    
    prompt_template_idea = ChatPromptTemplate.from_messages([
        ("system", "あなたは新しいアイデアを生み出すのが得意なAIです。与えられたトピックに関して、ユニークで面白いアイデアを一つ提案してください。アイデアは簡潔に一行で述べてください。"),
        ("human", "トピック: {topic}")
    ])
    
    chain = prompt_template_idea | llm 
    response = chain.invoke({"topic": topic})
    idea = response.content.strip()
    
    print(f"idea_generation_node: Generated idea: '{idea}'")
    return {"generated_idea": idea, "messages": [AIMessage(content=f"Generated Idea: {idea}")]}

def idea_evaluation_node(state: MultiLLMState):
    idea = state["generated_idea"]
    if not idea:
        return {"messages": [AIMessage(content="No idea to evaluate.")], "evaluated_idea": "N/A"}
        
    print(f"idea_evaluation_node: Evaluating idea: '{idea}'")
    
    prompt_template_eval = ChatPromptTemplate.from_messages([
        ("system", "あなたはアイデアを客観的に評価するのが得意なAIです。与えられたアイデアについて、その実現可能性と面白さを評価し、短いコメントを述べてください。"),
        ("human", "評価対象のアイデア: {idea}")
    ])
    
    chain = prompt_template_eval | llm 
    response = chain.invoke({"idea": idea})
    evaluation = response.content.strip()
    
    print(f"idea_evaluation_node: Evaluation: '{evaluation}'")
    return {"evaluated_idea": evaluation, "messages": [AIMessage(content=f"Evaluation: {evaluation}")]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(MultiLLMState)

workflow.add_node("capture_topic", get_topic)
workflow.add_node("generate_idea", idea_generation_node)
workflow.add_node("evaluate_idea", idea_evaluation_node)

workflow.set_entry_point("capture_topic")

workflow.add_edge("capture_topic", "generate_idea")
workflow.add_edge("generate_idea", "evaluate_idea")
workflow.add_edge("evaluate_idea", END)

app = workflow.compile()

# --- グラフの実行と結果表示 ---
topics_to_test = [
    "新しい料理のレシピ",
    "未来の交通手段",
    "週末の過ごし方"
]

for topic_text in topics_to_test:
    print(f"\n--- 複数LLM連携テスト (トピック: {topic_text}) ---")
    inputs = {
        "messages": [HumanMessage(content=topic_text)],
        # original_topic, generated_idea, evaluated_idea はグラフ内で設定されるので初期値はNoneや空文字でOK
        "original_topic": "", 
        "generated_idea": None,
        "evaluated_idea": None
    }
    final_state = app.invoke(inputs, {"recursion_limit": 5})
    print(f"Final State for topic '{topic_text}':")
    print(f"  Original Topic: {final_state.get('original_topic')}")
    print(f"  Generated Idea: {final_state.get('generated_idea')}")
    print(f"  Evaluated Idea: {final_state.get('evaluated_idea')}")

try:
    display(Image(app.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")
``````
</details>

<details><summary>解説013</summary>

#### この問題のポイント
*   **学習内容:** 一つのグラフ内に、それぞれ異なるプロンプトや役割を持つ複数のLLM呼び出しノードを配置し、それらを連携させる方法を学びます。これにより、より複雑で多段階の思考や処理を行うエージェントやパイプラインを構築できます。
*   **コード解説:**
    *   `MultiLLMState`には、ユーザーからの最初のトピック (`original_topic`)、最初のLLMが生成したアイデア (`generated_idea`)、そして二番目のLLMが評価した結果 (`evaluated_idea`) を保持するキーが定義されています。
    *   `get_topic`ノード: ユーザーの入力を `original_topic` として状態に保存します。
    *   `idea_generation_node`: `original_topic` に基づいて、アイデア生成用のプロンプト (`prompt_template_idea`) を使用してLLMを呼び出し、結果を `generated_idea` に保存します。
    *   `idea_evaluation_node`: `generated_idea` に基づいて、アイデア評価用のプロンプト (`prompt_template_eval`) を使用してLLMを呼び出し（ここでも同じ `llm` インスタンスを使用していますが、プロンプトが異なるため役割が変わります）、結果を `evaluated_idea` に保存します。
    *   グラフは `capture_topic` -> `generate_idea` -> `evaluate_idea` -> `END` という直列な流れで、各ステップで状態が更新されていきます。
    *   各LLM呼び出しノード内では、`langchain_core.prompts.ChatPromptTemplate` を使ってそのノード専用のプロンプトを定義し、共通の `llm` インスタンスと組み合わせて (例: `chain = prompt_template | llm`) LLM呼び出しを行っています。これにより、同じLLMモデルでも異なる指示を与えることで、多様な処理を実現できます。
*   **応用例:**
    *   リサーチアシスタント: 質問受け付け -> 情報検索プロンプトでLLM -> 要約プロンプトでLLM -> 報告書作成プロンプトでLLM。
    *   コード生成・レビュー: 要件定義 -> コード生成LLM -> 生成コード評価LLM -> 修正指示LLM。
    *   このように、タスクを細分化し、各サブタスクに特化したプロンプトを持つLLMノードを連携させることで、より高品質な結果を得ることが期待できます。
---
</details>

### ■ 問題014: エラーハンドリングの準備 (シンプルなtry-except)

グラフ内のノード処理中にエラーが発生する可能性は常にあります（例: 外部API呼び出しの失敗、予期しないデータ形式など）。この問題では、ノード関数内で基本的な`try-except`ブロックを使い、エラーを捕捉して状態にエラー情報を記録し、処理を継続または安全に終了させる方法の基礎を学びます。これは第2章でより詳しく扱う「エラーハンドリング」の準備となります。

In [ ]:
# 解答欄014
from typing import TypedDict, Annotated, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
class ErrorHandlingState(TypedDict):
    messages: Annotated[list, add_messages]
    input_value: str
    processed_value: Optional[int] # 処理結果 (数値のはず)
    error_info: Optional[str]      # エラー発生時の情報

# --- ノード定義 (Nodes) ---
def risky_converter_node(state: ErrorHandlingState):
    raw_value = state["input_value"]
    print(f"risky_converter_node: Attempting to convert '{raw_value}' to integer.")
    
    try:
        # ここでエラーが発生する可能性のある処理
        converted_int = int(raw_value)
        print(f"  Successfully converted to {converted_int}")
        return {
            "processed_value": ____, # converted_int
            "error_info": None, # エラーなし
            "messages": [AIMessage(content=f"Successfully converted '{raw_value}' to {converted_int}.")]
        }
    except ValueError as e:
        # エラーを捕捉した場合
        error_message = f"Conversion failed: {str(e)}"
        print(f"  Error: {error_message}")
        return {
            "processed_value": None,
            "error_info": ____, # error_message
            "messages": [AIMessage(content=f"Error converting '{raw_value}': {error_message}")]
        }

def result_router(state: ErrorHandlingState):
    # error_info があればエラー処理へ、なければ成功処理へ
    if state.get("error_info"):
        print("result_router: Error detected, routing to error_handler.")
        return "handle_error"
    else:
        print("result_router: No error, routing to success_logger.")
        return "log_success"

def success_log_node(state: ErrorHandlingState):
    log_msg = f"Success! Processed value: {state['processed_value']}"
    print(f"success_log_node: {log_msg}")
    return {"messages": [AIMessage(content=log_msg)]}

def error_handler_node(state: ErrorHandlingState):
    log_msg = f"Handling error: {state['error_info']}. Input was '{state['input_value']}'."
    print(f"error_handler_node: {log_msg}")
    # ここでさらにエラーに応じた処理（例: デフォルト値設定、ユーザー通知など）も可能
    return {"messages": [AIMessage(content=log_msg)]}

# --- グラフ構築 (Graph) ---
workflow = ____(ErrorHandlingState) # StateGraph

workflow.____("converter", risky_converter_node) # add_node
workflow.____("success_logger", success_log_node) # add_node
workflow.____("error_handler", error_handler_node) # add_node

workflow.____("converter") # set_entry_point

workflow.____( # add_conditional_edges
    "converter",
    result_router,
    {
        "log_success": "success_logger",
        "handle_error": "error_handler"
    }
)

workflow.____("success_logger", ____) # add_edge, END
workflow.____("error_handler", ____) # add_edge, END

app = workflow.____() # compile

# --- グラフの実行と結果表示 ---
test_inputs_map = {
    "Valid Input": {"input_value": "123"},
    "Invalid Input": {"input_value": "abc"},
    "Empty Input": {"input_value": ""}
}

for test_name, data in test_inputs_map.items():
    print(f"\n--- エラーハンドリングテスト: {test_name} (Input: '{data['input_value']}') ---")
    # messages は add_messages のために空リストで初期化
    current_data = data.copy()
    current_data.setdefault("messages", [])
    current_data.setdefault("processed_value", None)
    current_data.setdefault("error_info", None)

    final_state = app.invoke(current_data, {"recursion_limit": 5})
    print(f"Final State for {test_name}: {final_state}")
    if final_state.get("error_info"):
        print(f"  Error Info: {final_state['error_info']}")
    else:
        print(f"  Processed Value: {final_state.get('processed_value')}")

try:
    display(Image(app.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")

<details><summary>解答014</summary>

``````python
# 解答014
from typing import TypedDict, Annotated, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class ErrorHandlingState(TypedDict):
    messages: Annotated[list, add_messages]
    input_value: str
    processed_value: Optional[int] # 処理結果 (数値のはず)
    error_info: Optional[str]      # エラー発生時の情報

# --- ノード定義 (Nodes) ---
def risky_converter_node(state: ErrorHandlingState):
    raw_value = state["input_value"]
    print(f"risky_converter_node: Attempting to convert '{raw_value}' to integer.")
    
    try:
        converted_int = int(raw_value)
        print(f"  Successfully converted to {converted_int}")
        return {
            "processed_value": converted_int,
            "error_info": None, 
            "messages": [AIMessage(content=f"Successfully converted '{raw_value}' to {converted_int}.")]
        }
    except ValueError as e:
        error_message = f"Conversion failed: {str(e)}"
        print(f"  Error: {error_message}")
        return {
            "processed_value": None,
            "error_info": error_message,
            "messages": [AIMessage(content=f"Error converting '{raw_value}': {error_message}")]
        }

def result_router(state: ErrorHandlingState):
    if state.get("error_info"):
        print("result_router: Error detected, routing to error_handler.")
        return "handle_error"
    else:
        print("result_router: No error, routing to success_logger.")
        return "log_success"

def success_log_node(state: ErrorHandlingState):
    log_msg = f"Success! Processed value: {state['processed_value']}"
    print(f"success_log_node: {log_msg}")
    return {"messages": [AIMessage(content=log_msg)]}

def error_handler_node(state: ErrorHandlingState):
    log_msg = f"Handling error: {state['error_info']}. Input was '{state['input_value']}'."
    print(f"error_handler_node: {log_msg}")
    return {"messages": [AIMessage(content=log_msg)]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(ErrorHandlingState)

workflow.add_node("converter", risky_converter_node)
workflow.add_node("success_logger", success_log_node)
workflow.add_node("error_handler", error_handler_node)

workflow.set_entry_point("converter")

workflow.add_conditional_edges(
    "converter",
    result_router,
    {
        "log_success": "success_logger",
        "handle_error": "error_handler"
    }
)

workflow.add_edge("success_logger", END)
workflow.add_edge("error_handler", END)

app = workflow.compile()

# --- グラフの実行と結果表示 ---
test_inputs_map = {
    "Valid Input": {"input_value": "123"},
    "Invalid Input": {"input_value": "abc"},
    "Empty Input": {"input_value": ""}
}

for test_name, data in test_inputs_map.items():
    print(f"\n--- エラーハンドリングテスト: {test_name} (Input: '{data['input_value']}') ---")
    current_data = data.copy()
    current_data.setdefault("messages", [])
    current_data.setdefault("processed_value", None)
    current_data.setdefault("error_info", None)

    final_state = app.invoke(current_data, {"recursion_limit": 5})
    print(f"Final State for {test_name}: {final_state}")
    if final_state.get("error_info"):
        print(f"  Error Info: {final_state['error_info']}")
    else:
        print(f"  Processed Value: {final_state.get('processed_value')}")

try:
    display(Image(app.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")
``````
</details>

<details><summary>解説014</summary>

#### この問題のポイント
*   **学習内容:** ノード処理中の潜在的なエラー（この例では文字列から整数への型変換失敗）を`try-except`ブロックで捕捉し、その結果（成功か失敗か、エラー情報など）をグラフの状態(`State`)に記録する方法を学びます。さらに、状態に記録されたエラー情報に基づいて、条件付きエッジを使って処理を分岐させ、成功時とエラー時で異なる後続処理を行う方法も理解します。
*   **コード解説:**
    *   `ErrorHandlingState`には、処理対象の入力 `input_value`、処理後の値 `processed_value`、そしてエラー情報を格納する `error_info` が定義されています。
    *   `risky_converter_node`: `input_value` を整数に変換しようとします。
        *   成功した場合: `processed_value` に変換後の整数を、`error_info` に `None` を設定して返します。
        *   `ValueError` が発生した場合 (例: "abc"を整数に変換しようとした): `except` ブロックでエラーを捕捉し、`processed_value` を `None` に、`error_info` にエラーメッセージ文字列を設定して返します。
    *   `result_router`ルーター関数: `state['error_info']` が存在するかどうか（つまり、エラーが発生したかどうか）を確認します。
        *   エラーがあれば `"handle_error"` を返し、`error_handler_node` に処理を分岐させます。
        *   エラーがなければ `"log_success"` を返し、`success_log_node` に処理を分岐させます。
    *   `success_log_node`: 成功メッセージをログに記録します。
    *   `error_handler_node`: エラー発生時の情報をログに記録します。実際のアプリケーションでは、ここでエラーに応じたフォールバック処理やユーザーへの通知などを行うことができます。
    *   グラフは、入力 -> `converter` -> (成功なら `success_logger` / エラーなら `error_handler`) -> `END` という流れになります。
*   **重要性:**
    *   堅牢なアプリケーションを構築するためには、予期せぬエラーへの対処が不可欠です。LangGraphにおいても、各ノードの処理でエラーが発生する可能性を考慮し、適切にハンドリングすることで、グラフ全体の安定性を高めることができます。
    *   この問題で示した方法は基本的なエラーハンドリングですが、より高度なエラー処理（リトライ、サーキットブレーカーなど）もLangGraphのループ構造や条件分岐を応用して実装可能です（これらは第2章以降のトピックとなります）。
---
</details>

### ■ 問題015: 第1章のまとめ - 簡単なQ&Aボットの改善

第1章で学んだ様々な要素（状態管理、LLM連携、条件分岐、ループ、情報抽出など）を組み合わせて、問題004で作成したシンプルなチャットボットを改善してみましょう。このQ&Aボットは、ユーザーの質問のタイプ（例: 単純な挨拶、知識を問う質問、不明な質問）を判別し、応答を変化させたり、会話の回数をカウントしたりする機能を持つようにします。

In [ ]:
# 解答欄015
from typing import TypedDict, Annotated, Literal, Optional
import re
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
QuestionCategory = Literal["greeting", "knowledge_q", "opinion_q", "unknown_q"]

class AdvancedQABotState(TypedDict):
    messages: Annotated[list, add_messages]
    user_input: str # 最新のユーザー入力
    question_category: Optional[QuestionCategory] # 質問のカテゴリ
    llm_response: Optional[str] # LLMからの最終的な応答
    conversation_turns: int # 会話のターン数
    max_turns: int # 最大許容ターン数
    error_message: Optional[str]

# --- ノード定義 (Nodes) ---
def capture_input_and_increment_turn(state: AdvancedQABotState):
    user_message = state["messages"][-1].content
    current_turns = state.get("conversation_turns", 0) + 1
    print(f"capture_input_and_increment_turn: User: '{user_message}', Turn: {current_turns}")
    return {
        "user_input": user_message,
        "conversation_turns": current_turns,
        "question_category": None, # 各処理の前に初期化
        "llm_response": None, # 各処理の前に初期化
        "error_message": None # 各処理の前に初期化
    }

def categorize_question_node(state: AdvancedQABotState):
    text = state["user_input"].lower()
    category: QuestionCategory = "unknown_q"
    
    # 簡単なキーワードベースのカテゴリ分け（LLMを使っても良い）
    if any(greet in text for greet in ["こんにちは", "やあ", "どうも"]):
        category = "greeting"
    elif any(q_word in text for q_word in ["何ですか", "教えて", "とは", "なぜ"]) or "?" in text:
        # ここでは知識を問う質問と意見を問う質問を単純に分けるのは難しいので、一旦「知識」としておく
        category = "knowledge_q"
        if any(opinion_word in text for opinion_word in ["どう思う", "意見は"]):
             category = "opinion_q"
    
    print(f"categorize_question_node: Input '{text}' categorized as '{category}'")
    return {"question_category": category, "messages": [AIMessage(content=f"Category: {category}")]}

def greeting_responder_node(state: AdvancedQABotState):
    response = "こんにちは！ご用件は何でしょうか？"
    print(f"greeting_responder_node: Responding with '{response}'")
    return {"llm_response": response}

def knowledge_llm_node(state: AdvancedQABotState):
    question = state["user_input"]
    print(f"knowledge_llm_node: Asking LLM (knowledge): '{question}'")
    # 知識ベースの質問応答用プロンプト (問題013のように専用プロンプトも可)
    # ここではシンプルにそのまま質問
    response_obj = llm.invoke([HumanMessage(content=question)])
    response_text = response_obj.content.strip()
    print(f"  LLM response: {response_text}")
    return {"llm_response": response_text}

def opinion_llm_node(state: AdvancedQABotState):
    question = state["user_input"]
    print(f"opinion_llm_node: Asking LLM (opinion): '{question}'")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "あなたは様々なトピックについて個人的な意見を述べることができるAIです。ただし、客観的な事実と意見を区別して話してください。"),
        ("human", "{user_question}")
    ])
    chain = prompt | llm
    response_obj = chain.invoke({"user_question": question})
    response_text = response_obj.content.strip()
    print(f"  LLM response: {response_text}")
    return {"llm_response": response_text}

def unknown_question_node(state: AdvancedQABotState):
    response = "申し訳ありません、よく理解できませんでした。別の言葉で質問していただけますか？"
    print(f"unknown_question_node: Responding with '{response}'")
    return {"llm_response": response}

def final_response_node(state: AdvancedQABotState):
    # 最終的な応答をmessagesに追加
    final_resp = state.get("llm_response", "(No response generated)")
    print(f"final_response_node: Final AI response: '{final_resp}'")
    return {"messages": [AIMessage(content=final_resp)]}

# --- ルーター関数 ---
def route_by_category(state: AdvancedQABotState):
    category = state.get("question_category")
    print(f"route_by_category: Routing based on category '{category}'")
    if category == "greeting":
        return "greeting_responder"
    elif category == "knowledge_q":
        return "knowledge_llm"
    elif category == "opinion_q":
        return "opinion_llm"
    else: # unknown_q or None
        return "unknown_responder"

def check_conversation_limit(state: AdvancedQABotState):
    current_turns = state.get("conversation_turns", 0)
    max_t = state.get("max_turns", 3)
    if current_turns >= max_t:
        print(f"check_conversation_limit: Max turns ({max_t}) reached. Ending conversation.")
        # 最後の応答を生成するために、一旦 final_response_node には行かせる
        # もし、ここで完全に終了させたいなら、ENDに直接つなぐノードを用意する
        return "__end__" # 直接ENDへ
    print(f"check_conversation_limit: Turn {current_turns}/{max_t}. Continuing.")
    return "continue_conversation" # 通常はカテゴリ分類へ

# --- グラフ構築 (Graph) ---
workflow = ____(AdvancedQABotState) # StateGraph

workflow.add_node("input_handler", capture_input_and_increment_turn)
workflow.add_node("categorizer", categorize_question_node)
workflow.add_node("greeting_responder", greeting_responder_node)
workflow.add_node("knowledge_llm", knowledge_llm_node)
workflow.add_node("opinion_llm", opinion_llm_node)
workflow.add_node("unknown_responder", unknown_question_node)
workflow.add_node("final_responder", final_response_node)

workflow.set_entry_point("input_handler")

# 入力後、会話ターン数チェック
workflow.add_conditional_edges(
    "input_handler",
    check_conversation_limit,
    {
        "continue_conversation": "categorizer",
        "__end__": END # 最大ターン数に達したら終了
    }
)

# カテゴリ分類後、各処理ノードへ
workflow.add_conditional_edges(
    "categorizer",
    route_by_category,
    {
        "greeting_responder": "greeting_responder",
        "knowledge_llm": "knowledge_llm",
        "opinion_llm": "opinion_llm",
        "unknown_responder": "unknown_responder"
    }
)

# 各応答生成ノードから最終応答整形ノードへ
workflow.add_edge("greeting_responder", "final_responder")
workflow.add_edge("knowledge_llm", "final_responder")
workflow.add_edge("opinion_llm", "final_responder")
workflow.add_edge("unknown_responder", "final_responder")

# 最終応答整形後、終了
workflow.add_edge("final_responder", END)

app = workflow.compile()

# --- グラフの実行と結果表示 ---
chat_history = []
max_total_turns = 3 # このデモでの最大会話ターン数

print(f"--- Q&Aボット改善テスト (最大{max_total_turns}ターン) ---")
for turn in range(max_total_turns * 2): # ユーザー入力の機会を多めに用意
    user_q = input(f"You (Turn {turn//2 + 1}): ")
    if user_q.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break
    
    chat_history.append(HumanMessage(content=user_q))
    inputs = {
        "messages": chat_history,
        "max_turns": max_total_turns 
        # conversation_turns は capture_input_and_increment_turn で初期化/更新
    }
    
    try:
        final_bot_state = app.invoke(inputs, {"recursion_limit": 15})
        if final_bot_state and final_bot_state.get("messages"):
            bot_response_message = final_bot_state["messages"][-1]
            if isinstance(bot_response_message, AIMessage):
                print(f"Bot: {bot_response_message.content}")
                chat_history.append(bot_response_message)
            else: # ENDに直接到達した場合など、AIMessageがない場合
                print("(Bot ended conversation due to max turns or other condition)")
                if final_bot_state.get("llm_response"):
                     print(f"Bot (last intended response): {final_bot_state['llm_response']}") # END直前の応答を表示試行
                break 
        else:
            print("(No response from bot or bot ended)")
            break

        # conversation_turns が max_turns に達したか、それ以上ならループを抜ける
        if final_bot_state.get("conversation_turns", 0) >= final_bot_state.get("max_turns", max_total_turns):
            print("Max conversation turns reached from bot's perspective.")
            break
            
    except Exception as e:
        print(f"Error during bot invocation: {e}")
        break

print("\n--- Final Chat History ---")
for msg in chat_history:
    print(f"  {msg.type.upper()}: {msg.content}")

try:
    display(Image(app.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")

<details><summary>解答015</summary>

``````python
# 解答015
from typing import TypedDict, Annotated, Literal, Optional
import re
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import Image, display

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
QuestionCategory = Literal["greeting", "knowledge_q", "opinion_q", "unknown_q"]

class AdvancedQABotState(TypedDict):
    messages: Annotated[list, add_messages]
    user_input: str # 最新のユーザー入力
    question_category: Optional[QuestionCategory] # 質問のカテゴリ
    llm_response: Optional[str] # LLMからの最終的な応答
    conversation_turns: int # 会話のターン数
    max_turns: int # 最大許容ターン数
    error_message: Optional[str]

# --- ノード定義 (Nodes) ---
def capture_input_and_increment_turn(state: AdvancedQABotState):
    user_message = state["messages"][-1].content
    # messages以外のキーは、invokeの入力で与えられなければ、前のターンの値が残る。
    # conversation_turns は入力で初期値0を渡すか、ここで .get(key, default) を使う。
    current_turns = state.get("conversation_turns", 0) + 1
    print(f"capture_input_and_increment_turn: User: '{user_message}', Turn: {current_turns}")
    return {
        "user_input": user_message,
        "conversation_turns": current_turns,
        "question_category": None, 
        "llm_response": None, 
        "error_message": None 
    }

def categorize_question_node(state: AdvancedQABotState):
    text = state["user_input"].lower()
    category: QuestionCategory = "unknown_q"
    
    if any(greet in text for greet in ["こんにちは", "やあ", "どうも", "hello", "hi"]):
        category = "greeting"
    elif any(q_word in text for q_word in ["とは", "何ですか", "教えて", "なぜ", "what is", "tell me about", "why"]) or "?" in text:
        if any(opinion_word in text for opinion_word in ["どう思う", "あなたの意見は", "what do you think about"]):
             category = "opinion_q"
        else:
            category = "knowledge_q"
    
    print(f"categorize_question_node: Input '{text}' categorized as '{category}'")
    return {"question_category": category, "messages": [AIMessage(content=f"Category determination: {category}")]}

def greeting_responder_node(state: AdvancedQABotState):
    response = "こんにちは！何かお手伝いできることはありますか？"
    print(f"greeting_responder_node: Responding with '{response}'")
    return {"llm_response": response}

def knowledge_llm_node(state: AdvancedQABotState):
    question = state["user_input"]
    print(f"knowledge_llm_node: Asking LLM (knowledge): '{question}'")
    response_obj = llm.invoke([HumanMessage(content=question)])
    response_text = response_obj.content.strip()
    print(f"  LLM response: {response_text}")
    return {"llm_response": response_text}

def opinion_llm_node(state: AdvancedQABotState):
    question = state["user_input"]
    print(f"opinion_llm_node: Asking LLM (opinion): '{question}'")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "あなたは様々なトピックについて個人的な意見を述べることができるAIです。客観的な事実とあなたの意見を区別して話してください。"),
        ("human", "{user_question}")
    ])
    chain = prompt | llm
    response_obj = chain.invoke({"user_question": question})
    response_text = response_obj.content.strip()
    print(f"  LLM response: {response_text}")
    return {"llm_response": response_text}

def unknown_question_node(state: AdvancedQABotState):
    response = "申し訳ありませんが、ご質問の意図を正確に理解できませんでした。もう少し具体的に、または別の言葉で質問していただけますでしょうか？"
    print(f"unknown_question_node: Responding with '{response}'")
    return {"llm_response": response}

def final_response_node(state: AdvancedQABotState):
    final_resp = state.get("llm_response", "(AI did not generate a response for some reason)")
    print(f"final_response_node: Final AI response to be added to messages: '{final_resp}'")
    return {"messages": [AIMessage(content=final_resp)]}

# --- ルーター関数 ---
def route_by_category(state: AdvancedQABotState):
    category = state.get("question_category")
    print(f"route_by_category: Routing based on category '{category}'")
    if category == "greeting": return "greeting_responder"
    if category == "knowledge_q": return "knowledge_llm"
    if category == "opinion_q": return "opinion_llm"
    return "unknown_responder"

def check_conversation_limit(state: AdvancedQABotState):
    current_turns = state.get("conversation_turns", 0)
    max_t = state.get("max_turns", 3) # デフォルトの最大ターン数を設定
    if current_turns >= max_t:
        print(f"check_conversation_limit: Max turns ({max_t}) reached for this interaction. Ending conversation.")
        # 最後の応答をユーザーに返すために、ENDの前に final_response_node を挟むこともできるが、
        # ここでは、最大ターンに達したら即座に終了するロジックとする。
        # その場合、最後のユーザー入力に対する応答は生成されない。
        # もし「最大ターンに達したので終了します」というメッセージを返したい場合は、
        # このルーターから専用の終了メッセージ生成ノードに繋ぎ、そこからENDへ。
        return "__end__" 
    print(f"check_conversation_limit: Turn {current_turns}/{max_t}. Continuing to categorize.")
    return "continue_to_categorizer"

# --- グラフ構築 (Graph) ---
workflow = StateGraph(AdvancedQABotState)

workflow.add_node("input_handler", capture_input_and_increment_turn)
workflow.add_node("categorizer", categorize_question_node)
workflow.add_node("greeting_responder", greeting_responder_node)
workflow.add_node("knowledge_llm", knowledge_llm_node)
workflow.add_node("opinion_llm", opinion_llm_node)
workflow.add_node("unknown_responder", unknown_question_node)
workflow.add_node("final_responder", final_response_node)

workflow.set_entry_point("input_handler")

workflow.add_conditional_edges(
    "input_handler",
    check_conversation_limit,
    {
        "continue_to_categorizer": "categorizer",
        "__end__": END 
    }
)

workflow.add_conditional_edges(
    "categorizer",
    route_by_category,
    {
        "greeting_responder": "greeting_responder",
        "knowledge_llm": "knowledge_llm",
        "opinion_llm": "opinion_llm",
        "unknown_responder": "unknown_responder"
    }
)

workflow.add_edge("greeting_responder", "final_responder")
workflow.add_edge("knowledge_llm", "final_responder")
workflow.add_edge("opinion_llm", "final_responder")
workflow.add_edge("unknown_responder", "final_responder")
workflow.add_edge("final_responder", END)

app = workflow.compile()

# --- グラフの実行と結果表示 (インタラクティブテスト) ---
chat_history_for_bot = []
max_dialogue_turns = 3 # このインタラクティブセッションでの最大会話往復数

print(f"--- 第1章まとめ Q&Aボット (最大{max_dialogue_turns}往復) ---")
print("チャットを開始します。'exit' または 'quit' で終了します。")

for i in range(max_dialogue_turns):
    user_text = input(f"あなた (Turn {i + 1}): ")
    if user_text.lower() in ["exit", "quit"]:
        print("チャットを終了します。")
        break
    
    chat_history_for_bot.append(HumanMessage(content=user_text))
    
    # conversation_turnsはグラフ内で管理されるが、max_turnsは外部から指定
    # messages以外のキーは、前回のinvokeの出力状態が引き継がれるが、
    # 毎回クリーンな状態で始めたい場合は、ここで初期値を設定する。
    # この例では、conversation_turnsはグラフ内でインクリメントされる想定。
    current_invoke_inputs = {
        "messages": chat_history_for_bot,
        "max_turns": max_dialogue_turns, # グラフ内の会話ターン数上限
        "conversation_turns": i # 現在のinvoke呼び出しが何ターン目か（グラフ内部のカウンタとは別管理）
                                 # もしくは、グラフに初期ターン数を渡すなら conversation_turns: 0 のようにする
                                 # ここでは capture_input_and_increment_turn で conversation_turns を
                                 # state.get("conversation_turns", 0) + 1 で更新するので、
                                 # invokeのたびに初期化されるか、前回の値が引き継がれる。
                                 # 簡単のため、毎回 fresh な conversation_turns で始める場合は
                                 # invoke の input に conversation_turns: 0 を含めるか、
                                 # capture_input_and_increment_turn の中で messages の長さなどから判断する。
                                 # ここでは、デモとして conversation_turns はグラフ内でインクリメントされる想定
                                 # ただし、max_turns は invoke 時に渡す必要がある。
    }
    if i == 0: # 最初のターンのみ conversation_turns を明示的に0に設定
        current_invoke_inputs["conversation_turns"] = 0

    try:
        # streamで途中経過を見る場合
        # print("\nBot thinking...")
        # for event in app.stream(current_invoke_inputs, {"recursion_limit": 25}):
        #    print(f"  Stream event: {event}")
        # final_bot_state = event[list(event.keys())[-1]] # streamの最後のイベントのデータ部分

        final_bot_state = app.invoke(current_invoke_inputs, {"recursion_limit": 25})

        if final_bot_state and final_bot_state.get("messages"):
            # messagesリストの最後のAIメッセージを取得
            ai_messages = [m for m in final_bot_state["messages"] if isinstance(m, AIMessage)]
            if ai_messages:
                # 最後のAIメッセージ（final_responderによって追加されたもの）を表示
                bot_actual_response = ai_messages[-1].content 
                print(f"AIボット: {bot_actual_response}")
                chat_history_for_bot.append(AIMessage(content=bot_actual_response))
            else:
                print("AIボット: (応答がありませんでした。最大ターン数に達した可能性があります)")
                break # AIからの応答がない場合はループ終了
        else:
            print("AIボット: (状態取得エラーまたは会話終了)")
            break

        # グラフ内部の会話ターン数が上限に達したら終了
        if final_bot_state.get("conversation_turns", 0) >= final_bot_state.get("max_turns", max_dialogue_turns):
             print("(最大会話ターン数に達しました)")
             break
            
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        import traceback
        traceback.print_exc()
        break
else: # forループが正常に終了した場合 (breakされなかった場合)
    print("\n最大会話往復数に達したのでチャットを終了します。")

print("\n--- 最終的なチャット履歴 (ボットとの対話) ---")
for msg in chat_history_for_bot:
    print(f"  {msg.type.upper()}: {msg.content}")

try:
    display(Image(app.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}")
``````
</details>

<details><summary>解説015</summary>

#### この問題のポイント
*   **学習内容:** 第1章で学んだ複数の概念（`StateGraph`の定義、`TypedDict`による状態管理、ノードとエッジの追加、LLM呼び出し、条件付きエッジによる分岐、ループ（会話ターン数制限による間接的なループ制御）、状態キーの更新）を統合し、少し複雑な対話型のQ&Aボットを構築します。これにより、LangGraphの基本的な要素を組み合わせて実用的なアプリケーションを作成する流れを体験します。
*   **コード解説:**
    *   **`AdvancedQABotState`**: ユーザー入力、質問カテゴリ、LLM応答、会話ターン数、最大ターン数、エラーメッセージなど、多様な情報を保持する状態を定義します。
    *   **ノード群:**
        *   `capture_input_and_increment_turn`: ユーザー入力を取得し、会話ターン数をインクリメント。他の関連状態も初期化。
        *   `categorize_question_node`: ユーザー入力を簡単なルールでカテゴリ分け（挨拶、知識質問、意見質問、不明）。
        *   `greeting_responder_node`, `knowledge_llm_node`, `opinion_llm_node`, `unknown_question_node`: 各カテゴリに応じた応答を生成するノード。知識・意見質問ではLLMを呼び出します（`opinion_llm_node`では専用プロンプトを使用）。
        *   `final_response_node`: 生成された応答を最終的に`messages`状態に追加し、ユーザーに見える形にします。
    *   **ルーター関数:**
        *   `check_conversation_limit`: 会話ターン数が上限（`max_turns`）に達していたらグラフを終了させます。達していなければカテゴリ分類に進みます。
        *   `route_by_category`: `question_category`状態に基づいて、適切な応答生成ノードへ処理を分岐させます。
    *   **グラフ構造:**
        1.  エントリーポイントは `input_handler`。
        2.  `input_handler`の後、`check_conversation_limit`ルーターでターン数上限をチェック。上限なら`END`、そうでなければ`categorizer`へ。
        3.  `categorizer`の後、`route_by_category`ルーターで質問カテゴリに応じて各応答生成ノード（`greeting_responder`など）へ分岐。
        4.  各応答生成ノードは、処理後に`final_responder`へ遷移。
        5.  `final_responder`が最終応答を`messages`に追加し、その後`END`へ遷移してグラフの1回の実行が終了。
    *   **実行部分:** `input()`関数を使ってユーザーと対話形式でテストします。ユーザーが"exit"か"quit"を入力するか、最大会話往復数に達するまでループします。`invoke`のたびに、`messages`（会話履歴）と`max_turns`を渡しています。`conversation_turns`はグラフ内部でインクリメント・チェックされます。
*   **第1章の総括:**
    *   この問題を通じて、LangGraphの基本的ながらも強力な機能を一通り組み合わせる体験ができました。
    *   状態(`State`)がグラフ全体の情報伝達と制御のハブとして機能すること、ノードが個々の処理単位であること、エッジ（特に条件付きエッジ）が処理の流れを柔軟に定義することを実感できたでしょう。
    *   LLMの組み込みも、プロンプトを工夫することで様々な役割を担わせることが可能です。
    *   第2章以降では、ここでの基礎を元に、より高度な制御フロー（自己修正ループ、エラーハンドリング、人間による介入など）を学んでいきます。
---
</details>